<a href="https://colab.research.google.com/github/chayn1v/temp_lightgbm/blob/main/dataset_prep_prefix_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Files, Cleaning and some EDA


In [1]:
try:
  drive
  print('Drive Mounted already')
except:
  NameError
  from google.colab import drive
  drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import sys
 
import pandas as pd
import numpy as np
from itertools import product
import pandas_profiling
import gc
from tqdm import notebook  
import matplotlib.pyplot as plt
 

In [ ]:
import re
from os.path import commonprefix

In [ ]:
def downcast_dtypes(df):
    ''' provided for efficiency (in class)
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int16`
    '''
    before=df.memory_usage(index=True).sum()/1024**2
    print('original dataframe size', before,' MBs')
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype in [ "float64","float32"]]
    int_cols =   [c for c in df if df[c].dtype in ["int64","int32"]]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    after=df.memory_usage(index=True).sum()/1024**2
    reduction=after/before
    print('downcasted  dataframe  size', after,' MBs')
    print('fraction of original memory size: ', reduction)
    print(df.dtypes)
    return df    
  
 


In [ ]:
 
path='drive/My Drive/Colab Notebooks/week_3/'
# loading files 
sales = pd.read_csv(path+'sales_train_v2.csv', encoding='utf8')
items = pd.read_csv(path+'items.csv', encoding='utf8')   
shops = pd.read_csv(path+'shops.csv', encoding='utf8')
item_cats=pd.read_csv(path+'item_categories.csv', encoding='utf8')  
test=pd.read_csv(path+'test.csv', encoding='utf8')
#cons_conf_ind=pd.read_csv(path+'cons_conf_ind.csv', encoding='utf8')

#custom processed:
path1='drive/My Drive/'
item_categ_p=pd.read_csv(path1+'item_categ_p.csv', encoding='utf8')
item_categ_p=item_categ_p.rename(columns={"игр_all": "games_all"})
shops2=pd.read_csv(path1+'shops2.csv', encoding='utf8')
shops2=shops2.drop('shop_name',axis=1)
s_items1=pd.read_csv(path1+'s_items1.csv', encoding='utf8')

#downcast
datasets=[sales, items,shops, item_cats, test, item_categ_p,shops2,s_items1 ] 
for df in datasets:
  df=downcast_dtypes(df)

del datasets
gc.collect()  
   
#print(s_items1.head())
#print(s_items1.sparse.density)

s_items1_cols= list(s_items1.columns.values)
print(s_items1_cols)

original dataframe size 134.3926239013672  MBs
downcasted  dataframe  size 50.39731025695801  MBs
fraction of original memory size:  0.3750005676944396
date               object
date_block_num      int16
shop_id             int16
item_id             int16
item_price        float16
item_cnt_day      float16
dtype: object
original dataframe size 0.5075531005859375  MBs
downcasted  dataframe  size 0.25383758544921875  MBs
fraction of original memory size:  0.5001202537353816
item_name           object
item_id              int16
item_category_id     int16
dtype: object
original dataframe size 0.00103759765625  MBs
downcasted  dataframe  size 0.00069427490234375  MBs
fraction of original memory size:  0.6691176470588235
shop_name    object
shop_id       int16
dtype: object
original dataframe size 0.00140380859375  MBs
downcasted  dataframe  size 0.00092315673828125  MBs
fraction of original memory size:  0.657608695652174
item_category_name    object
item_category_id       int16
dtype: obje

In [ ]:
 # output format
pd.set_option('display.max_rows', 150)

pd.set_option('display.max_columns', 150)

In [ ]:
number_of_items_in_sales_per_month=sales.groupby('date_block_num').agg(number_of_items_in_sales =('item_id', 'nunique') )
number_of_items_in_test=test ['item_id'].nunique()
total_number_of_items_in_sales=sales ['item_id'].nunique() 



items_in_test=set(test ['item_id'].unique() )
items_in_sales=set(sales ['item_id'].unique() )
items_in_test_not_in_sales=items_in_test-items_in_sales


print('number_of_items_in_test',number_of_items_in_test)
print('items_in_test_not_in_sales',len(items_in_test_not_in_sales))
#print('number_of_items_in_sales_per_month',number_of_items_in_sales_per_month)
print('number_of_items_in_sales_total',total_number_of_items_in_sales )
print('items_in_test_not_in_sales +number_of_items_in_sales_total',len(items_in_test_not_in_sales) +total_number_of_items_in_sales )
print('Total items in items.csv: ',items['item_id'].nunique())

number_of_items_in_test 5100
items_in_test_not_in_sales 363
number_of_items_in_sales_total 21807
items_in_test_not_in_sales +number_of_items_in_sales_total 22170
Total items in items.csv:  22170


In [ ]:
   
#Editing the Kaggle test set to the same format as sales 
test1=test.drop(['ID'], axis=1)
test1['date' ]='01.11.2016'
test1['date_block_num' ]=34
#dud values not to be used
test1[ 'item_price'  ]=1
test1[  'item_cnt_day']=1
print(test1.head(3))
#required column order: date  date_block_num  shop_id  item_id  item_price  item_cnt_day 

sales=pd.concat([sales,test1], sort=True , ignore_index=True)
print(sales.loc[sales.date_block_num==34].head(3).T)
print(sales.loc[sales.date_block_num==33].head(3).T)

########################################################################
index_cols=['shop_id', 'item_id', 'date_block_num']
sales1=pd.concat([sales,test1], ignore_index=True, sort=False, keys=index_cols )
print(sales1.loc[sales1.date_block_num==34].head(3).T)
print(sales1.loc[sales1.date_block_num==33].head(3).T)


   shop_id  item_id        date  date_block_num  item_price  item_cnt_day
0        5     5037  01.11.2016              34           1             1
1        5     5320  01.11.2016              34           1             1
2        5     5233  01.11.2016              34           1             1
                   2935849     2935850     2935851
date            01.11.2016  01.11.2016  01.11.2016
date_block_num          34          34          34
item_cnt_day             1           1           1
item_id               5037        5320        5233
item_price               1           1           1
shop_id                  5           5           5
                   2882335     2882336     2882337
date            23.10.2015  05.10.2015  02.10.2015
date_block_num          33          33          33
item_cnt_day             1           1           1
item_id              13315       13880       13881
item_price             649         229         659
shop_id                 45          45   

In [ ]:
 items_=items[items.item_id.isin(items_in_test_not_in_sales)]
 

In [ ]:
#custom processing the data, creating features

def sales1_ (sales, exclude=False, only_test_shops=False):
  '''corrects duplicate shops, modifies sales''' 
  #sales1=sales.copy()

  # shop 11 has the same address as shop 10, data exists for only one month and is an error (or alternative name for the same shop) 
  sales.loc[(sales.shop_id==11) &(sales.date_block_num==25), 'shop_id']=10
  #We remove rows where data is reported yearly instead of monthly (in order to create item life features)
   

# Identifying shops with the same address
# 0      !Якутск Орджоникидзе, 56 фран
# 1      !Якутск ТЦ "Центральный" фран
# 57      Якутск Орджоникидзе, 56
# 58      Якутск ТЦ "Центральный"
#This may be considered as one shop (different businesses in 1 moll) NOT IMPLEMENTED
#23                  москва тк буденовский пав а2                 10     0
#24                  москва тк буденовский пав к7                 10     0
  for months in [0,1]:
    for shopp in [ 57,58 ]:
      sales.loc[(sales.shop_id==shopp-57) &(sales.date_block_num==months), 'shop_id']=shopp
  
# Excluding the shops which provided only one month per year data
#This may change the the count of items (some items exclusively belong to this shop))
  if exclude==True:
    sales=sales.loc[(sales.shop_id != 9) &(sales.shop_id != 20)]
     
  
  if only_test_shops==True:  #Default is false
    sales=sales.loc[(sales.shop_id != 9) &(sales.shop_id != 20)]
    sales=sales.loc[sales.shop_id.isin(test_shops) ]
    test_shops=np.sort(test.drop_duplicates(subset='shop_id').shop_id.values)
    test_shops=list(test_shops)
    print(test_shops )
    print(len(test_shops) )
  
  return sales 

 

#clipping and transforming price 
sales.item_price=sales.item_price.clip(1,100000)
#sales.item_price=np.log( sales.item_price+1)


In [ ]:
sales=sales1_ (sales,exclude=False)

#clipping and transforming price 
sales.item_price=sales.item_price.clip(1,100000)
#sales.item_price=np.log( sales.item_price+1)


print(test.head)

<bound method NDFrame.head of            ID  shop_id  item_id
0           0        5     5037
1           1        5     5320
2           2        5     5233
3           3        5     5232
4           4        5     5268
...       ...      ...      ...
214195  17587       45    18454
214196  17588       45    16188
214197  17589       45    15757
214198  17590       45    19648
214199  17591       45      969

[214200 rows x 3 columns]>


In [ ]:
#An example of shop which just opened and has no significant sales this month

print(sales.loc[(sales.shop_id==34)&(sales.date_block_num==18)])
#x=sales.loc[(sales.shop_id==34)&(sales.date_block_num==18)].index 
#sales=sales.drop(x) 

               date  date_block_num  item_cnt_day  item_id  item_price  \
1803319  31.07.2014              18           1.0    20949         5.0   
1803320  31.07.2014              18           1.0    11030       149.0   

         shop_id  
1803319       34  
1803320       34  


In [ ]:
#Locating all shop/months with less than 20 sales
gb=sales.groupby(['shop_id','date_block_num']).agg(target_shop=("item_cnt_day","sum")).target_shop.sort_values()

gb=gb[gb.le(20)]
print(gb[gb.le(20)] )
print(gb[gb.le(20)].index   )



shop_id  date_block_num
33       27               -1.0
27       32               -1.0
8        3                -1.0
9        27               -1.0
34       18                2.0
Name: target_shop, dtype: float64
MultiIndex([(33, 27),
            (27, 32),
            ( 8,  3),
            ( 9, 27),
            (34, 18)],
           names=['shop_id', 'date_block_num'])


In [ ]:
for (shop,month) in list(gb[gb.le(20)].index.values):
   ind=sales.loc[(sales.shop_id==shop)&(sales.date_block_num==month)].index 
   print(sales.loc[(sales.shop_id==shop)&(sales.date_block_num==month)])
   sales=sales.drop(ind)


               date  date_block_num  item_cnt_day  item_id  item_price  \
2561462  24.04.2015              27          -1.0     6673      7992.0   

         shop_id  
2561462       33  
               date  date_block_num  item_cnt_day  item_id  item_price  \
2878882  30.09.2015              32          -1.0     9250      1300.0   

         shop_id  
2878882       27  
              date  date_block_num  item_cnt_day  item_id  item_price  shop_id
366475  01.04.2013               3          -1.0     4809       599.0        8
               date  date_block_num  item_cnt_day  item_id  item_price  \
2600214  22.04.2015              27          -1.0     5672      3190.0   

         shop_id  
2600214        9  
               date  date_block_num  item_cnt_day  item_id  item_price  \
1803319  31.07.2014              18           1.0    20949         5.0   
1803320  31.07.2014              18           1.0    11030       149.0   

         shop_id  
1803319       34  
1803320       34  


In [ ]:
print(sales.columns.values)

#history=sales.loc[sales.item_cnt_day>0][['item_id','date_block_num','date']].groupby('item_id').agg(item_start=('date_block_num', 'min'),item_start_date=('date', 'min') , item_end=('date_block_num', 'max'))
history=sales.loc[sales.item_cnt_day>0][['item_id','date_block_num' ]].groupby('item_id').agg(item_start=('date_block_num', 'min') , item_end=('date_block_num', 'max'))
history_shop=sales.loc[sales.item_cnt_day>0][['item_id','date_block_num','shop_id']].groupby(['item_id','shop_id']).agg(item_shop_start=('date_block_num', 'min') , item_shop_end=('date_block_num', 'max'))
print(history.head())
print(history_shop.head())

['date' 'date_block_num' 'item_cnt_day' 'item_id' 'item_price' 'shop_id']
         item_start  item_end
item_id                      
0                20        20
1                15        21
2                19        22
3                18        19
4                20        20
                 item_shop_start  item_shop_end
item_id shop_id                                
0       54                    20             20
1       55                    15             21
2       54                    19             22
3       54                    18             19
4       54                    20             20


In [ ]:
#process date feature Do 1 time!
sales.date.describe()
sales['date']=pd.to_datetime( sales['date'], format="%d.%m.%Y" ) 
sales.date.describe()


In [ ]:
print(sales)

              date  date_block_num  item_cnt_day  item_id  item_price  shop_id
0       2013-01-02               0           1.0    22154       999.0       59
1       2013-01-03               0           1.0     2552       899.0       25
2       2013-01-05               0          -1.0     2552       899.0       25
3       2013-01-06               0           1.0     2554      1709.0       25
4       2013-01-15               0           1.0     2555      1099.0       25
...            ...             ...           ...      ...         ...      ...
3150044 2016-11-01              34           1.0    18454         1.0       45
3150045 2016-11-01              34           1.0    16188         1.0       45
3150046 2016-11-01              34           1.0    15757         1.0       45
3150047 2016-11-01              34           1.0    19648         1.0       45
3150048 2016-11-01              34           1.0      969         1.0       45

[3150043 rows x 6 columns]


In [ ]:
#EDA  
#shops where each item is sold 
shops_for_item=sales[sales.date<'2016-11-01'].groupby(by='item_id', as_index=False).agg(active_shops_for_item=( 'shop_id' ,'unique') )




In [ ]:
#EDA  
#shops where each item is sold (sorted)
shops_for_item
foo=lambda x: np.sort(x)
shops_for_item.active_shops_for_item=shops_for_item.active_shops_for_item.apply(foo)

In [ ]:
shops_for_item=pd.merge(shops_for_item,items,on='item_id', how='left')
shops_for_item=pd.merge(shops_for_item,history,on='item_id', how='left')

In [ ]:
shops_for_item

,item_id,active_shops_for_item,item_name,item_category_id,item_start,item_end
0,0,[54],! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,20.0,20.0
1,1,[55],!ABBYY FineReader 12 Professional Edition Full...,76,15.0,21.0
2,2,[54],***В ЛУЧАХ СЛАВЫ (UNV) D,40,19.0,22.0
3,3,[54],***ГОЛУБАЯ ВОЛНА (Univ) D,40,18.0,19.0
4,4,[54],***КОРОБКА (СТЕКЛО) D,40,20.0,20.0
...,...,...,...,...,...,...
21802,22165,[55],"Ядерный титбит 2 [PC, Цифровая версия]",31,7.0,16.0
21803,22166,[55],Язык запросов 1С:Предприятия [Цифровая версия],54,9.0,34.0
21804,22167,"[2, 3, 4, 5, 6, 7, 10, 12, 13, 14, 15, 16, 17,...",Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,8.0,34.0
21805,22168,"[2, 12, 16, 42, 43]",Яйцо для Little Inu,62,0.0,8.0


In [ ]:
#newEDA
items_for_shop = sales[sales.date<'2016-11-01'].groupby(by='shop_id' ).agg(active_items_for_shop=( 'item_id' ,'unique') )
items_for_shop.index.values
 

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59])

In [ ]:
s={}
for x in items_for_shop.index.values.tolist():
  print(x)
  #print(items_for_shop.iloc[x])
  s[x]=set(items_for_shop.iloc[x].values[0].tolist())
  #print(s[x])

2
3
4
5
6
7
8
9
10
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57


IndexError: ignored

In [ ]:
print(len(s))
print(list(s.keys()))
print((s[2]))
print(len(s[2]))
df = pd.DataFrame(columns = list(s.keys()))  
shop_similarity=np.zeros((60,60))
print(df)
for x in list(s.keys()):
  for y in list(s.keys()):
    if x>=y:
      #print(len(s[x].intersection(s[y]))/(len(s[x])+len(s[y])))
      #df[x,y]=len(s[x].intersection(s[y]))/(len(s[x])+len(s[y]))
      #shop_similarity[x,y]=2*len(s[x].intersection(s[y]))/(len(s[x])+len(s[y]))
      shop_similarity[x,y]= len(s[x].intersection(s[y]))/min(len(s[x]),len(s[y]))
      
      shop_similarity[y,x]=shop_similarity[x,y]
      #print(A[x,y]) 

54
[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
{27, 28, 29, 30, 31, 32, 33, 35, 38, 40, 47, 48, 49, 51, 59, 62, 64, 67, 71, 72, 73, 74, 75, 76, 77, 78, 85, 87, 88, 90, 91, 94, 98, 109, 113, 115, 154, 197, 199, 213, 351, 385, 421, 464, 472, 482, 484, 485, 486, 491, 498, 527, 534, 545, 554, 555, 556, 560, 638, 642, 646, 651, 657, 677, 687, 699, 700, 701, 715, 716, 719, 784, 785, 786, 791, 792, 795, 804, 806, 819, 829, 835, 839, 840, 934, 935, 944, 947, 948, 949, 950, 951, 953, 956, 957, 958, 960, 961, 963, 965, 968, 969, 970, 971, 972, 975, 976, 979, 980, 981, 982, 984, 987, 990, 991, 992, 993, 996, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1012, 1013, 1015, 1016, 1017, 1018, 1019, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1030, 1031, 1032, 1034, 1035, 1036, 1037, 1062, 1063, 1065, 1067, 1073

In [ ]:
 
shop_similarity_df=pd.DataFrame(shop_similarity) 
shop_similarity_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,1.000000,0.746330,0.896566,0.799425,0.675223,0.507123,0.731417,0.597483,0.658376,0.0,0.770557,0.812149,0.754362,0.786701,0.837461,0.235925,0.835272,0.682994,0.633096,0.745793,0.956492,0.814475,0.855794,0.916678,0.738405,0.777671,0.956629,0.645953,0.880526,0.838623,0.811055,0.779070,0.730880,0.711725,0.769775,0.801714,0.713297,0.900534,0.690664,0.782734,0.716966,0.828978,0.769325,0.803347,0.835182,0.818169,0.776850,0.775619,0.744561,0.890956,0.000000,0.878506,0.929676,0.819948,0.726912,0.0,0.0,0.0
3,0.0,0.0,0.746330,1.000000,0.918125,0.824816,0.613410,0.535613,0.724810,0.609401,0.683042,0.0,0.812112,0.819452,0.768515,0.801101,0.878882,0.305630,0.865048,0.700734,0.565615,0.769904,0.976285,0.847544,0.883258,0.949322,0.760870,0.780068,0.980237,0.612307,0.909474,0.860678,0.846132,0.864341,0.731084,0.741107,0.803955,0.845777,0.740119,0.936194,0.702093,0.808018,0.753811,0.865330,0.809712,0.833620,0.874538,0.856296,0.794466,0.797290,0.788255,0.891022,0.000000,0.899492,0.953416,0.867448,0.763128,0.0,0.0,0.0
4,0.0,0.0,0.896566,0.918125,1.000000,0.890856,0.858041,0.627493,0.899736,0.812923,0.886125,0.0,0.888862,0.850133,0.954634,0.864212,0.868696,0.449062,0.854017,0.857376,0.821634,0.882898,0.939862,0.875121,0.842003,0.861842,0.895465,0.844302,0.903340,0.839345,0.973684,0.952663,0.865283,0.922481,0.855086,0.896411,0.931396,0.951040,0.898990,0.832912,0.903221,0.872836,0.897166,0.875287,0.894211,0.940930,0.949815,0.878160,0.863028,0.890566,0.915225,0.857119,0.000000,0.828129,0.854757,0.867929,0.902535,0.0,0.0,0.0
5,0.0,0.0,0.799425,0.824816,0.890856,1.000000,0.734940,0.539886,0.796498,0.652669,0.738952,0.0,0.774436,0.801601,0.825514,0.768979,0.844652,0.264075,0.831070,0.683185,0.662597,0.730308,0.957676,0.811302,0.851928,0.917900,0.776326,0.759762,0.959253,0.713853,0.917895,0.887036,0.805845,0.833333,0.705918,0.783578,0.830566,0.861689,0.793829,0.906500,0.777201,0.766917,0.800082,0.836042,0.797270,0.857741,0.887744,0.829736,0.751395,0.750424,0.812475,0.882125,0.000000,0.862964,0.927359,0.824642,0.808070,0.0,0.0,0.0
6,0.0,0.0,0.675223,0.613410,0.858041,0.734940,1.000000,0.107550,0.588790,0.536930,0.552645,0.0,0.675223,0.798848,0.416972,0.743845,0.762179,0.005362,0.796752,0.619172,0.547931,0.665794,0.941854,0.763751,0.837611,0.897328,0.726035,0.845993,0.931378,0.561551,0.136842,0.104895,0.735464,0.093023,0.737559,0.646412,0.256155,0.108935,0.611315,0.840754,0.613934,0.746464,0.652174,0.738083,0.679937,0.230487,0.251441,0.755369,0.771608,0.754845,0.656888,0.946045,0.000000,0.860660,0.906757,0.752750,0.650602,0.0,0.0,0.0
7,0.0,0.0,0.507123,0.535613,0.627493,0.539886,0.107550,1.000000,0.441595,0.658120,0.241453,0.0,0.527066,0.544872,0.438746,0.556980,0.571937,0.651475,0.626068,0.512108,0.113960,0.505698,0.785613,0.552707,0.642450,0.678063,0.480769,0.522792,0.7884

#Aggregations and Mean Encodings per Month

In [ ]:
 

#sales=sales.copy()

#Month feature [0,11] 
sales['month']=sales['date'].dt.month-1 
sales['day_of_mnth']=sales.date.dt.day 
sales=sales.drop('date', axis=1)

print(sales.dtypes)
print(shops.dtypes)
 
sales=pd.merge(sales, items[['item_category_id','item_id'] ], how='left', on='item_id'  ) 


sales = pd.merge(sales, shops2, how='left', on='shop_id') 
sales = pd.merge(sales, item_categ_p, how='left', on='item_category_id')
sales = pd.merge(sales, s_items1, how='left', on='item_id')

item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

print(items.shape)
print(item_category_mapping.shape)
print(sales[2:5].T)

date_block_num      int64
item_cnt_day      float64
item_id             int16
item_price        float64
shop_id             int16
month               int64
day_of_mnth         int64
dtype: object
shop_name    object
shop_id       int16
dtype: object
(22170, 3)
(22170, 2)
                       2       3       4
date_block_num       0.0     0.0     0.0
item_cnt_day        -1.0     1.0     1.0
item_id           2552.0  2554.0  2555.0
item_price         899.0  1709.0  1099.0
shop_id             25.0    25.0    25.0
month                0.0     0.0     0.0
day_of_mnth          5.0     6.0    15.0
item_category_id    58.0    58.0    56.0
shop_type            2.0     2.0     2.0
city_code           11.0    11.0    11.0
games_all            0.0     0.0     0.0
class1               0.0     0.0     0.0
class2               0.0     0.0     0.0
class4               3.0     3.0     3.0
cd_count             0.0     0.0     2.0
bd_count             0.0     0.0     0.0
dvd_count            0.0     0.

In [ ]:
extra_aggregates=True
#lags_for_extras=False
 
if extra_aggregates==True:
  print( item_categ_p.columns.values)
  #
  print(shops2.columns.values)
  new_columns=list((set(item_categ_p.columns.values) | set( shops2.columns.values))-set(['item_category_id' ,'shop_id']))
  print(new_columns)

['item_category_id' 'games_all' 'class1' 'class2' 'class4']
['shop_id' 'shop_type' 'city_code']
['shop_type', 'class1', 'games_all', 'city_code', 'class4', 'class2']


In [ ]:
print(sales.shape) 

#items_in_test=set(test ['item_id'].unique() )
items_in_sales_and_test1=set(sales ['item_id'].unique() )
#items_in_test_not_in_sales=items_in_test-items_in_sales
print('items in sales after modification +test_items',len(items_in_sales_and_test1))



items_in_test=set(test ['item_id'].unique() )
items_in_sales=set(sales ['item_id'].unique() )
items_in_test_not_in_sales=items_in_test-items_in_sales


items_not_present_in_modified_sales_and_test=items_in_test- items_in_sales_and_test1


print('items_not_present_in_modified_sales_and_test',len(items_not_present_in_modified_sales_and_test))

sales.head  
##print(sales.ix[ 17])
print(sales.loc[(sales.item_id==2574)])#&(sales.date_block_num)==0]
#sales=sales.rename(columns={"игр_all": "games_all"})
sales=downcast_dtypes(sales)







(3150043, 41)
items in sales after modification +test_items 22170
items_not_present_in_modified_sales_and_test 0
         date_block_num  item_cnt_day  item_id  item_price  shop_id  month  \
10                    0           2.0     2574       399.0       25      0   
11                    0           1.0     2574       399.0       25      0   
12                    0           1.0     2574       399.0       25      0   
13                    0           2.0     2574       399.0       25      0   
14                    0           1.0     2574       399.0       25      0   
...                 ...           ...      ...         ...      ...    ...   
3124963              34           1.0     2574         1.0       46     10   
3130063              34           1.0     2574         1.0       41     10   
3135163              34           1.0     2574         1.0       44     10   
3140263              34           1.0     2574         1.0       39     10   
3145363              34      

In [ ]:
gc.collect()

14

##Creating the grids for for data.
Using some helper functions

In [ ]:
 
#Generalized grid:
def grid_df1(source_df,index_cols_):
  # by 'date_block_num', for all  other columns in (such as ['shop_id', 'item_id'])  in source_df[date_block_num']
  #index_cols=['shop_id', 'item_id', 'date_block_num']
  cols=index_cols_.copy()
  #print('cols = ', cols)
  cols.remove('date_block_num')
  #print('cols = ', cols)
  grid = [] 
  
  for block_num in source_df['date_block_num'].unique():
    prep_vals=[]
    for col in cols:
      cur_col = source_df.loc[source_df['date_block_num'] == block_num, col].unique()
      prep_vals.append(cur_col)
    prep_vals.append([block_num])  
    grid.append(np.array(list(product(*prep_vals)),dtype='int16'))

  # Turn the grid into a dataframe
  grid_df = pd.DataFrame(np.vstack(grid), columns = index_cols_,dtype=np.int16)
  return grid_df


In [ ]:
def group_by_merge(groupby_cols, source_df, grid, aggregation,fill_zero=True):
  #for iterative merges to grid
   
  agg_0=aggregation[0]
  agg_1=aggregation[1]
  #print(source_df.head())
  
  gb = source_df.groupby(groupby_cols  ).agg(x=agg_1)
  #print(gb.head( ))
   
  db_merged = pd.merge(grid, gb, how='left', on=groupby_cols)
  del gb
  if fill_zero:
    db_merged.x=db_merged.x.fillna(0)
   
  #print(agg_0)
  grid=db_merged.rename(columns={"x": agg_0})
  gc.collect()
  return grid
 

def save_csv(df):
  df=downcast_dtypes(df)
  N=df.shape[0]//2
  print(df[N:N+10])
   
  df.to_csv(path+df.name+'.csv', encoding='utf8',index=False )
   

In [ ]:
def create_lags(df,index_cols, shift_range=[1,2,3,4,12], no_lags=[],drop=True):
  #index_cols=index_cols+['item_category_id'] should be included
  #index_cols=index_cols+['date_block_num'] should be included
 
  print("index_cols = ", index_cols)
  cols_to_rename = list(set(df.columns.difference(index_cols))-set(no_lags)) 
  print(cols_to_rename)
  for month_shift in notebook.tqdm(shift_range):
    train_shift = df[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    df = pd.merge(df, train_shift, on=index_cols, how='left')#.fillna(0)
  
  if 'target' in set(cols_to_rename):
    cols_to_rename.remove('target')
  if 'mean' in set(cols_to_rename):
    cols_to_rename.remove('target')  
  if 'days' in set(cols_to_rename):
    cols_to_rename.remove('days')  
  if drop==True:
    df=df.drop(cols_to_rename,axis=1)
    print('cols_to_rename (dropped)', cols_to_rename)
  return df 
  
def diff(new,old,eps=.1):
  return 2*(new-old)/(new+old+eps)

#cols_diff=['target','mean_price_month','std_price_month', 'target_shop','days','target_item' , 'shop_item_cat_target', 'shop_item_cat_target_per_day']
def difference(df,cols_diff):
  #creates differences of columns with lags 1 and 2
  for col in cols_diff:
    u='{}_diff'.format(col) 
    v='{}_lag_1'.format(col)
    w='{}_lag_2'.format(col)
    df[u]=diff(df[v],df[w])
  return df  
  
def is_nan_check(df,cols=None):
  if cols==None:
    cols=list(df.columns.values)

  for col in cols:
     
    print(col, ' is nan count ', pd.isnull(df[col]).sum(), ' ; mean ',pd.isnull(df[col]).mean())  

In [ ]:
#Executable

index_cols = ['shop_id', 'item_id', 'date_block_num']
grid1=grid_df1( sales,index_cols)
 
#grid1.name="grid1"



In [ ]:
gc.collect()
print(grid1.head())
#print(sales.head())
 

   shop_id  item_id  date_block_num
0       59    22154               0
1       59     2552               0
2       59     2554               0
3       59     2555               0
4       59     2564               0


In [ ]:
groupby_cols=['shop_id', 'item_id', 'date_block_num']
arg=["target",('item_cnt_day','sum')]

#res=group_by_merge(groupby_cols, source_df=sales, grid=grid,aggregations= {target:('item_cnt_day','sum')})
 

#gb = sales.groupby(index_cols  ).agg( exec("target=('item_cnt_day','sum')") )# THERE SHOULD NOT BE A FILLNA HERE!  
#db_merged = pd.merge(res, gb, how='left', on=index_cols)
arg_list=[["target",('item_cnt_day','sum')],
          ["mean_price_month",('item_price','mean')],
          ["min_price_month",('item_price','min')],
          ["max_price_month",( 'item_price','max')],
           ["std_price_month",('item_price', 'std')]
          ]

for arg in arg_list : 
  grid1=group_by_merge(groupby_cols, source_df=sales, grid=grid1,aggregation= arg,fill_zero=False)        

In [ ]:
grid1.target=grid1.target.fillna(0)
grid1.target=grid1.target.clip(0,20)

In [ ]:
grid1.loc[grid1['date_block_num']==33].shape 
print('unique shops in month 33: ',grid1.loc[grid1['date_block_num']==33].shop_id.nunique())
print(grid1.loc[grid1['date_block_num']==33].shop_id.value_counts().head())


unique shops in month 33:  44
59    5413
58    5413
26    5413
25    5413
24    5413
Name: shop_id, dtype: int64


In [ ]:
print(grid1.head(10))
is_nan_check(grid1)
#save_csv(grid1)
#Check
#grid1.groupby(index_cols).target.nunique().head(50)

   shop_id  item_id  date_block_num  target  mean_price_month  \
0       59    22154               0     1.0             999.0   
1       59     2552               0     0.0               NaN   
2       59     2554               0     0.0               NaN   
3       59     2555               0     0.0               NaN   
4       59     2564               0     0.0               NaN   
5       59     2565               0     0.0               NaN   
6       59     2572               0     0.0               NaN   
7       59     2573               0     0.0               NaN   
8       59     2574               0     2.0             399.0   
9       59     2593               0     0.0               NaN   

   min_price_month  max_price_month  std_price_month  
0            999.0            999.0              NaN  
1              NaN              NaN              NaN  
2              NaN              NaN              NaN  
3              NaN              NaN              NaN  
4        

In [ ]:
#saving a copy for creating some advanced features later:
grid_sim=grid1[['shop_id',	'item_id',	'date_block_num'	,'target'	,'mean_price_month'	]].copy()
grid_sim

,shop_id,item_id,date_block_num,target,mean_price_month
0,59,22154,0,1.0,999.0
1,59,2552,0,0.0,NaN
2,59,2554,0,0.0,NaN
3,59,2555,0,0.0,NaN
4,59,2564,0,0.0,NaN
...,...,...,...,...,...
11097207,45,18454,34,1.0,1.0
11097208,45,16188,34,1.0,1.0
11097209,45,15757,34,1.0,1.0
11097210,45,19648,34,1.0,1.0


In [ ]:
# mean encoding shop/month

index_cols=['shop_id', 'date_block_num']
grid_shop_month=grid_df1( sales,index_cols)


#from grid1
arg=["target_shop",('target','sum')]
grid_shop_month=group_by_merge(groupby_cols=index_cols, source_df=grid1.loc[grid1.target>0], grid=grid_shop_month,aggregation= arg)


#from sales
#need a more careful parameter for days: some shops have single returns or sales on weekends
#arg=["days",('day_of_mnth', 'nunique')]
 
#grid_shop_month=group_by_merge(groupby_cols=index_cols, source_df=sales, grid=grid_shop_month,aggregation= arg)

 
sales_count_day=sales.groupby(index_cols+['day_of_mnth'],as_index=False).item_cnt_day.sum()
print(sales_count_day)
active_days_per_shop=sales_count_day[sales_count_day.item_cnt_day >3] .groupby(index_cols,as_index=False).day_of_mnth.nunique()
active_days_per_shop.rename(columns={'day_of_mnth':'days'},inplace=True )
print(grid_shop_month)
#grid_shop_month=grid_shop_month.fillna(0)
   

       shop_id  date_block_num  day_of_mnth  item_cnt_day
0            2               0            1          24.0
1            2               0            2          76.0
2            2               0            3          96.0
3            2               0            4          80.0
4            2               0            5          72.0
...        ...             ...          ...           ...
47260       59              33           28          14.0
47261       59              33           29          14.0
47262       59              33           30          30.0
47263       59              33           31          56.0
47264       59              34            1        5100.0

[47265 rows x 4 columns]
      shop_id  date_block_num  target_shop
0          59               0       2003.0
1          25               0       7692.0
2          24               0       1742.0
3          23               0       1932.0
4          19               0       2988.0
...       ...       

In [ ]:
#active_days_per_shop[active_days_per_shop.shop_id==6]

In [ ]:
#the following shops are oddballs and are probably all internet shops. So they are open 24/7 (for months when they are active) and we should fix the days column for them to full months. 
#for shop 9 we have data for months 9,21,33; for shop 20 -- for 21,33.
shops.loc[shops.shop_id.isin([12,9,20])]

,shop_name,shop_id
9,Выездная Торговля,9
12,Интернет-магазин ЧС,12
20,"Москва ""Распродажа""",20


In [ ]:
active_days_per_shop[active_days_per_shop.shop_id.isin([12,9,20])]

,shop_id,date_block_num,days
212,9,9,4
213,9,21,4
214,9,33,4
250,12,0,19
251,12,1,22
252,12,2,24
253,12,3,24
254,12,4,23
255,12,5,22
256,12,6,24


In [ ]:
#Take a shop which is always open:  
value=active_days_per_shop[active_days_per_shop.shop_id==6].days.values
#subset days values for shop 12.
mask=(active_days_per_shop.shop_id==12)
mask_list=mask[mask==True].index.to_list()
print(mask_list)
print(len(mask_list))
active_days_per_shop.loc[mask_list,'days']=value
#(active_days_per_shop.loc[active_days_per_shop.shop_id==12])
#subset days values for shop 9 and 20.
for s in [9,20]:
  mask=(active_days_per_shop.shop_id==s)
  mask_list=mask[mask==True].index.to_list()
  active_days_per_shop.loc[mask_list,'days']=31

#fixed days values
#active_days_per_shop.loc[active_days_per_shop.shop_id.isin([9,12,20])]

 

[250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284]
35


In [ ]:
#quick check
#active_days_per_shop.loc[active_days_per_shop.shop_id.isin([9,12,20])]

In [ ]:
#fixing days variable for test month 34
mask=active_days_per_shop[(active_days_per_shop.date_block_num==34)]
mask_list=mask[mask==True].index.to_list()
active_days_per_shop.loc[mask_list,'days']=30
#active_days_per_shop[(active_days_per_shop.date_block_num==34)]


In [ ]:
grid_shop_month=pd.merge(grid_shop_month,active_days_per_shop, on=index_cols, how='left') 
print(grid_shop_month)
print(active_days_per_shop)
grid_shop_month

      shop_id  date_block_num  target_shop  days
0          59               0       2003.0    30
1          25               0       7692.0    30
2          24               0       1742.0    30
3          23               0       1932.0    30
4          19               0       2988.0    31
...       ...             ...          ...   ...
1618       46              34       5100.0    30
1619       41              34       5100.0    30
1620       44              34       5100.0    30
1621       39              34       5100.0    30
1622       45              34       5100.0    30

[1623 rows x 4 columns]
      shop_id  date_block_num  days
0           2               0    25
1           2               1    20
2           2               2    31
3           2               3    30
4           2               4    31
...       ...             ...   ...
1618       59              30    31
1619       59              31    31
1620       59              32    30
1621       59              

,shop_id,date_block_num,target_shop,days
0,59,0,2003.0,30
1,25,0,7692.0,30
2,24,0,1742.0,30
3,23,0,1932.0,30
4,19,0,2988.0,31
...,...,...,...,...
1618,46,34,5100.0,30
1619,41,34,5100.0,30
1620,44,34,5100.0,30
1621,39,34,5100.0,30


In [ ]:
#days_shop_36=sales.loc[(sales.date_block_num==33)&(sales.shop_id==36)].groupby('day_of_mnth').agg(t=('item_cnt_day',"sum"))
#print(days_shop_36)

In [ ]:
#no_lag=index_cols+days
grid_shop_month=create_lags( grid_shop_month,index_cols, shift_range=[1,2,3,4,12],no_lags=['days'])
grid_shop_month=grid_shop_month.fillna(0)
 
grid_shop_month['month']=grid_shop_month.date_block_num%12
grid_shop_month['year']=grid_shop_month.date_block_num//12
gc.collect()




index_cols =  ['shop_id', 'date_block_num']
['target_shop']



cols_to_rename (dropped) ['target_shop']


4

In [ ]:
grid_shop_month

,shop_id,date_block_num,days,target_shop_lag_1,target_shop_lag_2,target_shop_lag_3,target_shop_lag_4,target_shop_lag_12,month,year
0,59,0,30,0.0,0.0,0.0,0.0,0.0,0,0
1,25,0,30,0.0,0.0,0.0,0.0,0.0,0,0
2,24,0,30,0.0,0.0,0.0,0.0,0.0,0,0
3,23,0,30,0.0,0.0,0.0,0.0,0.0,0,0
4,19,0,31,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
1618,46,34,30,1258.0,1242.0,1561.0,1608.0,2018.0,10,2
1619,41,34,30,739.0,661.0,886.0,734.0,1182.0,10,2
1620,44,34,30,779.0,759.0,770.0,678.0,1156.0,10,2
1621,39,34,30,836.0,790.0,813.0,598.0,840.0,10,2


In [ ]:
cols_diff=['target_shop']  
grid_shop_month=difference(grid_shop_month,cols_diff)
is_nan_check(grid_shop_month)

shop_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
days  is nan count  0  ; mean  0.0
target_shop_lag_1  is nan count  0  ; mean  0.0
target_shop_lag_2  is nan count  0  ; mean  0.0
target_shop_lag_3  is nan count  0  ; mean  0.0
target_shop_lag_4  is nan count  0  ; mean  0.0
target_shop_lag_12  is nan count  0  ; mean  0.0
month  is nan count  0  ; mean  0.0
year  is nan count  0  ; mean  0.0
target_shop_diff  is nan count  0  ; mean  0.0


In [ ]:
grid_shop_month.name="grid_shop_month_v2"
save_csv(grid_shop_month)

#It is used and deleted later!


#grid_shop_month=downcast_dtypes(grid_shop_month)
#print(grid_shop_month.head())
#del grid_shop_month

original dataframe size 0.055721282958984375  MBs
downcasted  dataframe  size 0.04643440246582031  MBs
fraction of original memory size:  0.8333333333333334
shop_id                 int16
date_block_num          int16
days                    int16
target_shop_lag_1     float16
target_shop_lag_2     float16
target_shop_lag_3     float16
target_shop_lag_4     float16
target_shop_lag_12    float16
month                   int16
year                    int16
target_shop_diff      float16
dtype: object
     shop_id  date_block_num  days  target_shop_lag_1  target_shop_lag_2  \
811       54              17    29             4992.0             4624.0   
812       53              17    30             1605.0             1448.0   
813       50              17    30             1830.0             2027.0   
814       51              17    30             1218.0             1403.0   
815       49              17    30              688.0              579.0   
816       52              17    30         

In [ ]:
#item_month_aggs
 
index_cols=['item_id', 'date_block_num'] 
grid_item_month=grid_df1( sales,index_cols)

 
args=[["target_item",('target' , 'sum' )],["item_active_shop_count",('shop_id', 'nunique')]]
for arg in args:
  grid_item_month=group_by_merge(groupby_cols=index_cols, source_df=grid1.loc[grid1.target>0], grid=grid_item_month,aggregation= arg)

#grid_item_month["target_item"]=grid_item_month.target_item.fillna(0)
#grid_item_month['item_active_shop_count']=grid_item_month.item_active_shop_count.fillna(0)


grid_item_month['mean_item_per_active_shop']=grid_item_month.target_item/grid_item_month.item_active_shop_count 
#grid_item_month['mean_item_per_active_shop']=grid_item_month['mean_item_per_active_shop'].fillna(0)

grid_item_month=grid_item_month.fillna(0)

In [ ]:
#Creating a copy for advanced features
grid_item_month_for_pref=grid_item_month.copy()
grid_item_month_for_pref.loc[grid_item_month_for_pref.date_block_num==34]

,item_id,date_block_num,target_item,item_active_shop_count,mean_item_per_active_shop
233911,5037,34,42.0,42.0,1.0
233912,5320,34,42.0,42.0,1.0
233913,5233,34,42.0,42.0,1.0
233914,5232,34,42.0,42.0,1.0
233915,5268,34,42.0,42.0,1.0
...,...,...,...,...,...
239006,18454,34,42.0,42.0,1.0
239007,16188,34,42.0,42.0,1.0
239008,15757,34,42.0,42.0,1.0
239009,19648,34,42.0,42.0,1.0


In [ ]:
grid_item_month=create_lags( grid_item_month,index_cols, shift_range=[1,2,3,4,12])

grid_item_month=pd.merge(grid_item_month,history, how='left', on='item_id')

#history for each item
#we assume that  the new items in the test set appear because they were sold in one of the shops  so value 0 means 'sold this month for the first time somewhere'
grid_item_month=grid_item_month.fillna(0)

grid_item_month['history']=(grid_item_month['date_block_num']-grid_item_month['item_start' ]).clip(-1,35)
grid_item_month=grid_item_month.drop(['item_end','item_start'],axis=1)
is_nan_check(grid_item_month)

index_cols =  ['item_id', 'date_block_num']
['item_active_shop_count', 'mean_item_per_active_shop', 'target_item']



cols_to_rename (dropped) ['item_active_shop_count', 'mean_item_per_active_shop', 'target_item']
item_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
item_active_shop_count_lag_1  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_1  is nan count  0  ; mean  0.0
target_item_lag_1  is nan count  0  ; mean  0.0
item_active_shop_count_lag_2  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_2  is nan count  0  ; mean  0.0
target_item_lag_2  is nan count  0  ; mean  0.0
item_active_shop_count_lag_3  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_3  is nan count  0  ; mean  0.0
target_item_lag_3  is nan count  0  ; mean  0.0
item_active_shop_count_lag_4  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_4  is nan count  0  ; mean  0.0
target_item_lag_4  is nan count  0  ; mean  0.0
item_active_shop_count_lag_12  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_12  is nan count  0  ; mean  0.0
target_item_lag_12  i

In [ ]:
cols_diff=["target_item"]  
grid_item_month=difference(grid_item_month,cols_diff)
 

In [ ]:
is_nan_check(grid_item_month,cols=['target_item_diff'])
 

target_item_diff  is nan count  0  ; mean  0.0


In [ ]:
#print(grid_item_month.head(10))
grid_item_month.name="grid_item_month_v2"
save_csv(grid_item_month)
#print(grid_item_month.head(10))

#del grid_item_month
gc.collect()


#print(items)
#print(sales.head())

original dataframe size 25.529129028320312  MBs
downcasted  dataframe  size 10.485177993774414  MBs
fraction of original memory size:  0.4107142857142857
item_id                               int16
date_block_num                        int16
item_active_shop_count_lag_1        float16
mean_item_per_active_shop_lag_1     float16
target_item_lag_1                   float16
item_active_shop_count_lag_2        float16
mean_item_per_active_shop_lag_2     float16
target_item_lag_2                   float16
item_active_shop_count_lag_3        float16
mean_item_per_active_shop_lag_3     float16
target_item_lag_3                   float16
item_active_shop_count_lag_4        float16
mean_item_per_active_shop_lag_4     float16
target_item_lag_4                   float16
item_active_shop_count_lag_12       float16
mean_item_per_active_shop_lag_12    float16
target_item_lag_12                  float16
history                             float16
target_item_diff                    float16
dtype: obj

30

In [ ]:
#shop/month/category
#need to merge item_categories to grid1:
grid1=pd.merge(grid1, items[['item_id','item_category_id']], how='left', on='item_id')
#print(grid1.head(10))


In [ ]:

index_cols=[ 'shop_id', 'item_category_id','date_block_num' ]
grid_item_cat_month=grid_df1( sales,index_cols)

#print(grid_item_cat_month.shape )
#print(grid_item_cat_month.loc[grid_item_cat_month.shop_id==54])
arg_list=[["shop_item_cat_target",('target','sum')],
          ["shop_item_cat_mean_price",('mean_price_month','mean')]]

for arg in arg_list : 
  grid_item_cat_month=group_by_merge(groupby_cols=index_cols, source_df=grid1, grid=grid_item_cat_month,aggregation= arg ,fill_zero=False)

grid_item_cat_month.shop_item_cat_target=grid_item_cat_month.shop_item_cat_target.fillna(0)


to_merge=['shop_id','date_block_num']+['days']
grid_item_cat_month=pd.merge(grid_item_cat_month,grid_shop_month[to_merge],how="left",on=['shop_id','date_block_num'])
#This aggregate is to be done later (when the whole file is assembled)
grid_item_cat_month['shop_item_cat_target_per_day']=grid_item_cat_month.shop_item_cat_target/(grid_item_cat_month.days )
grid_item_cat_month['shop_item_cat_target_per_day']=grid_item_cat_month['shop_item_cat_target_per_day'].fillna(0)  

grid_item_cat_month=grid_item_cat_month.drop([  "days"],axis=1)

grid_item_cat_month=grid_item_cat_month.fillna(0)

In [ ]:
#print(grid1.head())
print(grid_item_cat_month.head())

   shop_id  item_category_id  date_block_num  shop_item_cat_target  \
0       59                37               0                  60.0   
1       59                58               0                   0.0   
2       59                56               0                   1.0   
3       59                59               0                   2.0   
4       59                55               0                 169.0   

   shop_item_cat_mean_price  shop_item_cat_target_per_day  
0                     454.0                      2.000000  
1                       0.0                      0.000000  
2                    1299.0                      0.033333  
3                    1274.0                      0.066667  
4                     271.5                      5.633333  


In [ ]:
grid_item_cat_month=create_lags( grid_item_cat_month,index_cols, shift_range=[1,2,3,4,12])
grid_item_cat_month=grid_item_cat_month.fillna(0)

cols_diff=[ 'shop_item_cat_target','shop_item_cat_target_per_day']  
grid_item_cat_month=difference(grid_item_cat_month,cols_diff)



is_nan_check(grid_item_cat_month)

index_cols =  ['shop_id', 'item_category_id', 'date_block_num']
['shop_item_cat_target_per_day', 'shop_item_cat_mean_price', 'shop_item_cat_target']



cols_to_rename (dropped) ['shop_item_cat_target_per_day', 'shop_item_cat_mean_price', 'shop_item_cat_target']
shop_id  is nan count  0  ; mean  0.0
item_category_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_1  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_1  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_1  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_2  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_2  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_2  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_3  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_3  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_3  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_4  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_4  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_4  is nan count  0  ; mean  0.0
shop_item_cat_target_per_da

In [ ]:
#N=grid_item_cat_month.shape[0]//2
#print( grid_item_cat_month[N:N+10])
grid_item_cat_month.name="grid_item_cat_month_v2"
#save_csv(grid_item_cat_month)
#print(grid_item_cat_month.head(10))

del grid_item_cat_month,grid_shop_month
gc.collect()
 

100

In [ ]:
s_items1_cols=s_items1.columns.values
print(s_items1_cols) 
s_items1_cols=[x for x in s_items1_cols if "text" in x]+['item_id']
 
print(s_items1_cols) 
#need to merge item text features to grid1:
#grid1=pd.merge(grid1, s_items1[s_items1_cols], how='left', on='item_id')
grid1=pd.merge(grid1, s_items1, how='left', on='item_id')
grid1=pd.merge(grid1,item_categ_p, how='left', on='item_category_id')

grid1=pd.merge(grid1,shops2, how='left', on='shop_id')

 
 


print(grid1.head(10))


  
  

['item_id' 'cd_count' 'bd_count' 'dvd_count' 'lp_count' 'code_item_id'
 'code_item_id1' 'text_feature_0' 'text_feature_1' 'text_feature_2'
 'text_feature_3' 'text_feature_4' 'text_feature_5' 'text_feature_6'
 'text_feature_7' 'text_feature_8' 'text_feature_9' 'text_feature_10'
 'text_feature_11' 'text_feature_12' 'text_feature_13' 'text_feature_14'
 'text_feature_15' 'text_feature_16' 'text_feature_17' 'text_feature_18'
 'text_feature_19' 'text_feature_20']
['text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20', 'item_id']
   shop_id  item_id  date_block_num  target  mean_price_month  \
0       59    22154               0     1.0             999.0   
1      

In [ ]:
gc.collect() 
print(shops2.columns.values)
print(item_categ_p.columns.values)
print( new_columns )


['shop_id' 'shop_type' 'city_code']
['item_category_id' 'games_all' 'class1' 'class2' 'class4']
['shop_type', 'class1', 'games_all', 'city_code', 'class4', 'class2']


In [ ]:
#text_features/month
s_items1_cols=s_items1.columns.values
s_items1_cols=[x for x in s_items1_cols if "text" in x]
print(s_items1_cols) 
print(type(new_columns),new_columns )
#grouplist=['shop_id']+s_items1_cols+[ 'date_block_num']  
grouplist=s_items1_cols+[ 'date_block_num']  
     
print(grouplist)

#Cannot use Grid function due to memory limitations
#grid_grouplist=grid_df1( sales,grouplist) 
 
 

  
 




['text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20']
<class 'list'> ['shop_type', 'class1', 'games_all', 'city_code', 'class4', 'class2']
['text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20', 'date_block_num']


In [ ]:
#Due to out of memory error we cannot use groupby_merge

#there is an error before save: item_id should not be there!!!

gb = grid1.loc[(grid1.target>0)&((grid1[s_items1_cols]>0).any(axis=1))].groupby(grouplist).agg(item_groupllist_target=('target' , 'sum' ),item_groupllist_count=('item_id','nunique'))

grid_grouplist = pd.merge(grid1, gb, how='left', on=grouplist)

grid_grouplist.item_groupllist_target=grid_grouplist.item_groupllist_target.fillna(0)
grid_grouplist.item_groupllist_count=grid_grouplist.item_groupllist_count.fillna(0)

grid_grouplist['ratio']=grid_grouplist.item_groupllist_target  /grid_grouplist.item_groupllist_count
grid_grouplist['ratio']=grid_grouplist['ratio'].fillna(0)

del gb
gc.collect()
 


 


0

In [ ]:
(grid1[s_items1_cols]>0).any(axis=1)

0            True
1            True
2            True
3            True
4            True
            ...  
11097207     True
11097208     True
11097209     True
11097210     True
11097211    False
Length: 11097212, dtype: bool

In [ ]:
grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False)

,shop_id,item_id,date_block_num,target,mean_price_month,min_price_month,max_price_month,std_price_month,item_category_id,cd_count,bd_count,dvd_count,lp_count,code_item_id,code_item_id1,text_feature_0,text_feature_1,text_feature_2,text_feature_3,text_feature_4,text_feature_5,text_feature_6,text_feature_7,text_feature_8,text_feature_9,text_feature_10,text_feature_11,text_feature_12,text_feature_13,text_feature_14,text_feature_15,text_feature_16,text_feature_17,text_feature_18,text_feature_19,text_feature_20,games_all,class1,class2,class4,shop_type,city_code,item_groupllist_target,item_groupllist_count,ratio
10499752,55,13834,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,11693,2613,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,949.0,163.0,5.822086
10619066,7,16443,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,13490,3046,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,3,4,949.0,163.0,5.822086
10451998,38,16494,32,1.0,169.0,169.0,169.0,NaN,40,0,0,0,0,13688,3071,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,14,949.0,163.0,5.822086
10451997,38,15836,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,14248,3287,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,14,949.0,163.0,5.822086
10451996,38,15833,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,14248,3287,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,14,949.0,163.0,5.822086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10551504,16,4418,32,0.0,NaN,NaN,NaN,NaN,55,0,0,0,0,3356,688,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,8,0.0,0.0,0.000000
10551505,16,4427,32,0.0,NaN,NaN,NaN,NaN,55,0,0,0,0,3353,688,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,8,0.0,0.0,0.000000
10551643,16,21416,32,0.0,NaN,NaN,NaN,NaN,37,0,1,0,0,16210,3993,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,4,8,0.0,0.0,0.000000
10551657,16,21872,32,0.0,NaN,NaN,NaN,NaN,37,0,1,0,0,17536,4366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,4,8,0.0,0.0,0.000000


In [ ]:
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].item_groupllist_count.value_counts())
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False).shape)

1.0      47124
0.0      39270
2.0      20244
3.0      10080
4.0       8736
163.0     6846
5.0       6132
122.0     5124
29.0      4872
6.0       4032
93.0      3906
7.0       3822
11.0      3696
21.0      3528
28.0      3528
76.0      3192
74.0      3108
10.0      2940
17.0      2856
16.0      2730
8.0       2688
63.0      2646
58.0      2436
14.0      2352
54.0      2268
13.0      2184
24.0      2016
48.0      2016
9.0       1890
34.0      1428
15.0      1260
25.0      1050
12.0      1008
22.0       924
20.0       840
19.0       798
Name: item_groupllist_count, dtype: int64
(213570, 45)


In [ ]:
#just to have a look how good this aggregations are 
#print(grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False)[80000:])
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].item_groupllist_count.value_counts())
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False).shape)

1.0      47124
0.0      39270
2.0      20244
3.0      10080
4.0       8736
163.0     6846
5.0       6132
122.0     5124
29.0      4872
6.0       4032
93.0      3906
7.0       3822
11.0      3696
21.0      3528
28.0      3528
76.0      3192
74.0      3108
10.0      2940
17.0      2856
16.0      2730
8.0       2688
63.0      2646
58.0      2436
14.0      2352
54.0      2268
13.0      2184
24.0      2016
48.0      2016
9.0       1890
34.0      1428
15.0      1260
25.0      1050
12.0      1008
22.0       924
20.0       840
19.0       798
Name: item_groupllist_count, dtype: int64
(213570, 45)


In [ ]:
grouplist_save_cols=[	'shop_id',	'item_id' ]+grouplist+['item_groupllist_target',	'item_groupllist_count',	'ratio']
grid_grouplist=grid_grouplist[grouplist_save_cols]
grid_grouplist.name="grid_grouplist"
print(grid_grouplist.head( ))

 


   shop_id  item_id  text_feature_0  text_feature_1  text_feature_2  \
0       59    22154               0               0               0   
1       59     2552             181              86               0   
2       59     2554             181               0               0   
3       59     2555             181               0               0   
4       59     2564             181               0               0   

   text_feature_3  text_feature_4  text_feature_5  text_feature_6  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               8               0               0              55   
4               0               0               0               0   

   text_feature_7  text_feature_8  text_feature_9  text_feature_10  \
0               0               0               0                0   
1               0 

In [ ]:
index_cols=[	'shop_id',	'item_id' ]+grouplist
lags=[1,2,3,4]
grid_grouplist=create_lags( grid_grouplist,index_cols, shift_range=lags)
grid_grouplist=grid_grouplist.fillna(0)
cols_diff=['item_groupllist_target','item_groupllist_count','ratio']  
grid_grouplist=difference(grid_grouplist,cols_diff)

index_cols =  ['shop_id', 'item_id', 'text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20', 'date_block_num']
['item_groupllist_target', 'ratio', 'item_groupllist_count']



cols_to_rename (dropped) ['item_groupllist_target', 'ratio', 'item_groupllist_count']


In [ ]:
#We don't need to save all features in the grid_grouplist file 
is_nan_check(grid_grouplist)
grouplist.remove("date_block_num")
print(grouplist)
grid_grouplist=grid_grouplist.drop(grouplist, axis=1)
print(grid_grouplist)

shop_id  is nan count  0  ; mean  0.0
item_id  is nan count  0  ; mean  0.0
text_feature_0  is nan count  0  ; mean  0.0
text_feature_1  is nan count  0  ; mean  0.0
text_feature_2  is nan count  0  ; mean  0.0
text_feature_3  is nan count  0  ; mean  0.0
text_feature_4  is nan count  0  ; mean  0.0
text_feature_5  is nan count  0  ; mean  0.0
text_feature_6  is nan count  0  ; mean  0.0
text_feature_7  is nan count  0  ; mean  0.0
text_feature_8  is nan count  0  ; mean  0.0
text_feature_9  is nan count  0  ; mean  0.0
text_feature_10  is nan count  0  ; mean  0.0
text_feature_11  is nan count  0  ; mean  0.0
text_feature_12  is nan count  0  ; mean  0.0
text_feature_13  is nan count  0  ; mean  0.0
text_feature_14  is nan count  0  ; mean  0.0
text_feature_15  is nan count  0  ; mean  0.0
text_feature_16  is nan count  0  ; mean  0.0
text_feature_17  is nan count  0  ; mean  0.0
text_feature_18  is nan count  0  ; mean  0.0
text_feature_19  is nan count  0  ; mean  0.0
text_feature_2

In [ ]:
#grid_grouplist.name="grid_grouplist"
grid_grouplist.name="grid_grouplist_lvl_3"
#save_csv(grid_grouplist)
del grid_grouplist
gc.collect()

55

In [ ]:
print(sales.columns.values)
print(grid1.columns.values)
print(s_items1_cols)

['date_block_num' 'item_cnt_day' 'item_id' 'item_price' 'shop_id' 'month'
 'day_of_mnth' 'item_category_id' 'shop_type' 'city_code' 'games_all'
 'class1' 'class2' 'class4' 'cd_count' 'bd_count' 'dvd_count' 'lp_count'
 'code_item_id' 'code_item_id1' 'text_feature_0' 'text_feature_1'
 'text_feature_2' 'text_feature_3' 'text_feature_4' 'text_feature_5'
 'text_feature_6' 'text_feature_7' 'text_feature_8' 'text_feature_9'
 'text_feature_10' 'text_feature_11' 'text_feature_12' 'text_feature_13'
 'text_feature_14' 'text_feature_15' 'text_feature_16' 'text_feature_17'
 'text_feature_18' 'text_feature_19' 'text_feature_20']
['shop_id' 'item_id' 'date_block_num' 'target' 'mean_price_month'
 'min_price_month' 'max_price_month' 'std_price_month' 'item_category_id'
 'cd_count' 'bd_count' 'dvd_count' 'lp_count' 'code_item_id'
 'code_item_id1' 'text_feature_0' 'text_feature_1' 'text_feature_2'
 'text_feature_3' 'text_feature_4' 'text_feature_5' 'text_feature_6'
 'text_feature_7' 'text_feature_8' 'tex

In [ ]:
#mean-encode text features: need to do something with zero values (when all features are zero?)(mean across all items or active items only? where the target is non-zero)
gc.collect()
counter=0
necessary_features=['date_block_num', 'shop_id','item_id','target']+s_items1_cols
grid_text_features=grid1[necessary_features]
for x in s_items1_cols:
  
  groupby_cols=['date_block_num', 'shop_id',x]
  feature='text_feature_{}_enc'.format(counter)
  gb=grid1.groupby(groupby_cols).agg(feature=("target","mean")) 
  grid_text_features = pd.merge(grid_text_features, gb, how='left', on=groupby_cols)
  grid_text_features.rename(columns={"feature": feature},inplace=True )
  counter+=1

#Downcast not needed!!  
  

In [ ]:
grid_text_features [1000:10010]
grid_text_features =grid_text_features.drop(s_items1_cols+['target'],axis=1)

In [ ]:
grid_text_features [1000:10010]

,date_block_num,shop_id,item_id,text_feature_0_enc,text_feature_1_enc,text_feature_2_enc,text_feature_3_enc,text_feature_4_enc,text_feature_5_enc,text_feature_6_enc,text_feature_7_enc,text_feature_8_enc,text_feature_9_enc,text_feature_10_enc,text_feature_11_enc,text_feature_12_enc,text_feature_13_enc,text_feature_14_enc,text_feature_15_enc,text_feature_16_enc,text_feature_17_enc,text_feature_18_enc,text_feature_19_enc,text_feature_20_enc
1000,0,59,8460,0.184082,0.164429,0.040985,0.204834,0.244629,0.002048,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1001,0,59,1958,0.184082,0.164429,0.790527,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.741211,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1002,0,59,1722,0.184082,0.399902,0.249756,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1003,0,59,3152,0.184082,0.164429,0.249756,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1004,0,59,3159,0.184082,0.164429,0.249756,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.620605,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10005,0,25,12652,0.861816,0.809570,0.953613,0.900879,0.942383,0.969238,0.939453,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707
10006,0,25,12653,0.861816,0.514160,0.643066,0.900879,0.942383,0.969238,0.695801,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707
10007,0,25,12654,0.861816,0.514160,0.643066,0.900879,0.942383,0.969238,0.695801,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707
10008,0,25,12658,0.391357,0.809570,0.953613,0.900879,0.942383,0.969238,0.695801,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707


In [ ]:

lags=[1,2]
index_cols=['date_block_num', 'shop_id','item_id']
grid_text_features=create_lags( grid_text_features,index_cols, shift_range=lags) 

index_cols =  ['date_block_num', 'shop_id', 'item_id']
['text_feature_4_enc', 'text_feature_16_enc', 'text_feature_18_enc', 'text_feature_12_enc', 'text_feature_5_enc', 'text_feature_14_enc', 'text_feature_3_enc', 'text_feature_20_enc', 'text_feature_0_enc', 'text_feature_10_enc', 'text_feature_7_enc', 'text_feature_17_enc', 'text_feature_9_enc', 'text_feature_19_enc', 'text_feature_1_enc', 'text_feature_8_enc', 'text_feature_15_enc', 'text_feature_11_enc', 'text_feature_6_enc', 'text_feature_2_enc', 'text_feature_13_enc']



cols_to_rename (dropped) ['text_feature_4_enc', 'text_feature_16_enc', 'text_feature_18_enc', 'text_feature_12_enc', 'text_feature_5_enc', 'text_feature_14_enc', 'text_feature_3_enc', 'text_feature_20_enc', 'text_feature_0_enc', 'text_feature_10_enc', 'text_feature_7_enc', 'text_feature_17_enc', 'text_feature_9_enc', 'text_feature_19_enc', 'text_feature_1_enc', 'text_feature_8_enc', 'text_feature_15_enc', 'text_feature_11_enc', 'text_feature_6_enc', 'text_feature_2_enc', 'text_feature_13_enc']


In [ ]:
grid_text_features [1000000:10010000]

,date_block_num,shop_id,item_id,text_feature_4_enc_lag_1,text_feature_16_enc_lag_1,text_feature_18_enc_lag_1,text_feature_12_enc_lag_1,text_feature_5_enc_lag_1,text_feature_14_enc_lag_1,text_feature_3_enc_lag_1,text_feature_20_enc_lag_1,text_feature_0_enc_lag_1,text_feature_10_enc_lag_1,text_feature_7_enc_lag_1,text_feature_17_enc_lag_1,text_feature_9_enc_lag_1,text_feature_19_enc_lag_1,text_feature_1_enc_lag_1,text_feature_8_enc_lag_1,text_feature_15_enc_lag_1,text_feature_11_enc_lag_1,text_feature_6_enc_lag_1,text_feature_2_enc_lag_1,text_feature_13_enc_lag_1,text_feature_4_enc_lag_2,text_feature_16_enc_lag_2,text_feature_18_enc_lag_2,text_feature_12_enc_lag_2,text_feature_5_enc_lag_2,text_feature_14_enc_lag_2,text_feature_3_enc_lag_2,text_feature_20_enc_lag_2,text_feature_0_enc_lag_2,text_feature_10_enc_lag_2,text_feature_7_enc_lag_2,text_feature_17_enc_lag_2,text_feature_9_enc_lag_2,text_feature_19_enc_lag_2,text_feature_1_enc_lag_2,text_feature_8_enc_lag_2,text_feature_15_enc_lag_2,text_feature_11_enc_lag_2,text_feature_6_enc_lag_2,text_feature_2_enc_lag_2,text_feature_13_enc_lag_2
1000000,2,8,9546,0.138916,0.068970,0.138916,0.142334,0.140625,0.139526,0.129028,0.142334,0.118896,0.138672,0.134399,0.138550,0.136108,0.139771,0.113159,0.141113,0.137939,0.134033,0.139282,0.143188,0.142456,0.178955,0.000000,0.172852,0.181030,0.176514,0.177002,0.154785,0.180054,0.138794,0.173828,0.171997,0.175903,0.171631,0.177368,0.132324,0.178345,0.172729,0.161743,0.178345,0.172241,0.182129
1000001,2,8,9539,0.138916,0.139160,0.138916,0.142334,0.140625,0.139526,0.129028,0.142334,0.118896,0.138672,0.134399,0.138550,0.136108,0.139771,0.113159,0.141113,0.241943,0.134033,0.139282,0.143188,0.172729,0.178955,0.176636,0.172852,0.181030,0.176514,0.177002,0.154785,0.180054,0.138794,0.173828,0.171997,0.175903,0.171631,0.177368,0.132324,0.178345,0.290283,0.161743,0.178345,0.172241,0.103760
1000002,2,8,6882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000003,2,8,6848,0.138916,0.139160,0.138916,0.142334,0.140625,0.139526,0.129028,0.142334,0.118896,0.138672,0.134399,0.138550,0.136108,0.139771,0.113159,0.141113,0.137939,0.134033,0.139282,0.143188,0.142456,0.178955,0.176636,0.172852,0.181030,0.176514,0.177002,0.154785,0.180054,0.138794,0.173828,0.171997,0.175903,0.171631,0.177368,0.132324,0.178345,0.172729,0.161743,0.178345,0.172241,0.182129
1000004,2,8,6818,0.138916,0.139160,0.138916,0.142334,0.140625,0.139526,0.129028,0.142334,0.118896,0.138672,0.134399,0.138550,0.136108,0.139771,0.113159,0.141113,0.137939,0.203735,0.139282,0.143188,0.142456,0.178955,0.176636,0.172852,0.181030,0.176514,0.177002,0.154785,0.180054,0.138794,0.173828,0.171997,0.175903,0.171631,0.177368,0.132324,0.178345,0.172729,0.500000,0.178345,0.172241,0.182129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10009995,30,58,15740,0.298340,0.297119,0.293213,0.299805,0.314941,0.279053,0.302246,0.294678,0.280518,0.297119,0.296875,0.297607,0.282715,0.294922,0.240967,0.285645,0.283691,0.284180,0.275146,0.311279,0.301270,0.256104,0.259766,0.255371,0.259277,0.272949,0.249512,0.244263,0.254395,0.248291,0.258789,0.260742,0.259033,0.247681,0.253662,0.211792,0.246338,0.247803,0.247803,0.245361,0.275391,0.257812
10009996,30,58,15591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256104,0.259766,0.255371,0.259277,0.272949,0.249512,0.244263,0.254395,0.248291,0.258789,0.108826,0.259033,0.247681,0.253662,0.211792,0.246338,0.247803,0.247803,0.245361,0.275391,0.257812
10009997,30,58,16237,0.298340,0.297119,0.293213,0.299805,0.314941,0.279053,0.323975,0.294678,0.280518,0.297119,0.296875,0.297607,0.282715,0.294922,0.240967,0.410156,0.283691,0.284180,0.275146,0.281982,0.301270,0.256104,0.259766,0.255371,0.259277,0.272949

In [ ]:
is_nan_check(grid_text_features[grid_text_features.date_block_num>1])

date_block_num  is nan count  0  ; mean  0.0
shop_id  is nan count  0  ; mean  0.0
item_id  is nan count  0  ; mean  0.0
text_feature_4_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_16_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_18_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_12_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_5_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_14_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_3_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_20_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_0_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_10_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_7_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_17_enc_lag_1 

In [ ]:
grid_text_features=grid_text_features.fillna(0)
grid_text_features.name="grid_text_features_enc_v2"
#save_csv(grid_text_features)
del grid_text_features
gc.collect()

772

In [ ]:
 
index_cols=['item_id', 'city_code', 'date_block_num']

grid_item_city=grid_df1( sales,index_cols) 




gb = grid1.loc[grid1.target>0].groupby(index_cols ).agg(target_item_city_code=('target' , 'sum' ),item_active_city_code_count=('shop_id', 'nunique')).fillna(0)
grid_item_city = pd.merge(grid_item_city, gb, how='left', on=index_cols) 

grid_item_city.target_item_city_code=grid_item_city.target_item_city_code.fillna(0)
grid_item_city.item_active_city_code_count=grid_item_city.item_active_city_code_count.fillna(0)
 
 
grid_item_city['mean_item_per_active_city_code']=grid_item_city.target_item_city_code/grid_item_city.item_active_city_code_count 
grid_item_city['mean_item_per_active_city_code']=grid_item_city['mean_item_per_active_city_code'].fillna(0)  

 

In [ ]:
grid_item_city=create_lags( grid_item_city,index_cols, shift_range=[1,2,3,4,12])

index_cols =  ['item_id', 'city_code', 'date_block_num']
['item_active_city_code_count', 'target_item_city_code', 'mean_item_per_active_city_code']



cols_to_rename (dropped) ['item_active_city_code_count', 'target_item_city_code', 'mean_item_per_active_city_code']


In [ ]:
is_nan_check(grid_item_city)

item_id  is nan count  0  ; mean  0.0
city_code  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
item_active_city_code_count_lag_1  is nan count  1273882  ; mean  0.21787410430739718
target_item_city_code_lag_1  is nan count  1273882  ; mean  0.21787410430739718
mean_item_per_active_city_code_lag_1  is nan count  1273882  ; mean  0.21787410430739718
item_active_city_code_count_lag_2  is nan count  1592196  ; mean  0.2723158639354513
target_item_city_code_lag_2  is nan count  1592196  ; mean  0.2723158639354513
mean_item_per_active_city_code_lag_2  is nan count  1592196  ; mean  0.2723158639354513
item_active_city_code_count_lag_3  is nan count  1895665  ; mean  0.32421865913945097
target_item_city_code_lag_3  is nan count  1895665  ; mean  0.32421865913945097
mean_item_per_active_city_code_lag_3  is nan count  1895665  ; mean  0.32421865913945097
item_active_city_code_count_lag_4  is nan count  2184120  ; mean  0.37355358557532986
target_item_city_code_lag_4  

In [ ]:
grid_item_city.loc[grid_item_city.date_block_num==25].head(20)
grid_item_city=grid_item_city.fillna(0)

In [ ]:
cols_diff=['target_item_city_code', 'item_active_city_code_count', 'mean_item_per_active_city_code']  
grid_item_city=difference(grid_item_city,cols_diff)
grid_item_city=grid_item_city.fillna(0)

In [ ]:
grid_item_city.name="grid_item_city"
#save_csv(grid_item_city)
print(grid_item_city.head( ))

del grid_item_city,gb
gc.collect()



   item_id  city_code  date_block_num  item_active_city_code_count_lag_1  \
0    22154         25               0                                0.0   
1    22154         11               0                                0.0   
2    22154         10               0                                0.0   
3    22154          9               0                                0.0   
4    22154          2               0                                0.0   

   target_item_city_code_lag_1  mean_item_per_active_city_code_lag_1  \
0                          0.0                                   0.0   
1                          0.0                                   0.0   
2                          0.0                                   0.0   
3                          0.0                                   0.0   
4                          0.0                                   0.0   

   item_active_city_code_count_lag_2  target_item_city_code_lag_2  \
0                                0.0     

4

In [ ]:
  
index_cols=['shop_id','code_item_id1','date_block_num']

grid_shop_code1_month=grid_df1( sales,index_cols) 
 
gb = grid1.loc[grid1.target>0].groupby(index_cols ).agg(target_shop_code1=('target' , 'sum' ),shop_code1_count=('shop_id', 'nunique')).fillna(0)
grid_shop_code1_month = pd.merge(grid_shop_code1_month, gb, how='left', on=index_cols) 


grid_shop_code1_month.target_shop_code1=grid_shop_code1_month.target_shop_code1.fillna(0)
grid_shop_code1_month.shop_code1_count=grid_shop_code1_month.shop_code1_count.fillna(0)
 

 
grid_shop_code1_month['mean_target_shop_code1']=grid_shop_code1_month.target_shop_code1/grid_shop_code1_month.shop_code1_count
grid_shop_code1_month['mean_target_shop_code1']=grid_shop_code1_month['mean_target_shop_code1'].fillna(0)  

grid_shop_code1_month=grid_shop_code1_month.fillna(0)

grid_shop_code1_month=create_lags( grid_shop_code1_month,index_cols, shift_range=[1,2,3,4,12])
grid_shop_code1_month.name="grid_shop_code1_month"

grid_shop_code1_month=grid_shop_code1_month.fillna(0)







index_cols =  ['shop_id', 'code_item_id1', 'date_block_num']
['target_shop_code1', 'mean_target_shop_code1', 'shop_code1_count']



cols_to_rename (dropped) ['target_shop_code1', 'mean_target_shop_code1', 'shop_code1_count']


In [ ]:
grid_shop_code1_month.name="grid_shop_code1_month"
#save_csv(grid_shop_code1_month)

del grid_shop_code1_month,gb
gc.collect() 

4

In [ ]:

grid1['rev_per_item']=grid1['target']*grid1['mean_price_month']

index_cols=[  'shop_id','date_block_num'] 
gb=grid1.groupby (index_cols).agg(shop_rev=('rev_per_item', 'sum')).fillna(0) 

grid1=pd.merge(grid1,gb,  how='left', on=index_cols) 

grid1.rev_per_item=grid1.rev_per_item.fillna(0)
grid1.shop_rev=grid1.shop_rev.fillna(0)

In [ ]:
print(grid1.columns.values)
to_drop=grouplist+['cd_count', 'bd_count', 'dvd_count' ,'lp_count','code_item_id', 'code_item_id1']
grid1=grid1.drop(to_drop, axis=1)
print(grid1.columns.values)
is_nan_check(grid1)

['shop_id' 'item_id' 'date_block_num' 'target' 'mean_price_month'
 'min_price_month' 'max_price_month' 'std_price_month' 'item_category_id'
 'cd_count' 'bd_count' 'dvd_count' 'lp_count' 'code_item_id'
 'code_item_id1' 'text_feature_0' 'text_feature_1' 'text_feature_2'
 'text_feature_3' 'text_feature_4' 'text_feature_5' 'text_feature_6'
 'text_feature_7' 'text_feature_8' 'text_feature_9' 'text_feature_10'
 'text_feature_11' 'text_feature_12' 'text_feature_13' 'text_feature_14'
 'text_feature_15' 'text_feature_16' 'text_feature_17' 'text_feature_18'
 'text_feature_19' 'text_feature_20' 'games_all' 'class1' 'class2'
 'class4' 'shop_type' 'city_code' 'rev_per_item' 'shop_rev']
['shop_id' 'item_id' 'date_block_num' 'target' 'mean_price_month'
 'min_price_month' 'max_price_month' 'std_price_month' 'item_category_id'
 'games_all' 'class1' 'class2' 'class4' 'shop_type' 'city_code'
 'rev_per_item' 'shop_rev']
shop_id  is nan count  0  ; mean  0.0
item_id  is nan count  0  ; mean  0.0
date_block

In [ ]:
#grid1=grid1.drop(['cd_count', 'bd_count', 'dvd_count' ,'lp_count','code_item_id', 'code_item_id1'], axis=1)

In [ ]:
gc.collect()
print(grid1.columns.values)
 

['shop_id' 'item_id' 'date_block_num' 'target' 'mean_price_month'
 'min_price_month' 'max_price_month' 'std_price_month' 'item_category_id'
 'games_all' 'class1' 'class2' 'class4' 'shop_type' 'city_code'
 'rev_per_item' 'shop_rev']


In [ ]:
grid1=grid1.drop( 'item_category_id', axis=1)
grid1=grid1.drop( ['shop_type', 'class1', 'games_all','city_code''class4', 'class2'], axis=1)
index_cols=['shop_id' ,'item_id' ,'date_block_num']

shift_range=[1,2,3,4,12]
grid1=create_lags( grid1,index_cols, shift_range=shift_range)
for x in ['target','rev_per_item', 'shop_rev']:
  for y in shift_range:
    col='{}_lag_{}'.format(x,y)
    grid1[col]=grid1[col].fillna(0)

cols_diff=['target','mean_price_month','std_price_month','rev_per_item', 'shop_rev']  
grid1=difference(grid1,cols_diff)

for x in ['target','rev_per_item', 'shop_rev']:
  col='{}_diff'.format(x)
  grid1[col]=grid1[col].fillna(0)


index_cols =  ['shop_id', 'item_id', 'date_block_num']
['shop_type', 'class1', 'games_all', 'shop_rev', 'std_price_month', 'city_code', 'rev_per_item', 'mean_price_month', 'target', 'max_price_month', 'min_price_month', 'class4', 'class2']



cols_to_rename (dropped) ['shop_type', 'class1', 'games_all', 'shop_rev', 'std_price_month', 'city_code', 'rev_per_item', 'mean_price_month', 'max_price_month', 'min_price_month', 'class4', 'class2']


In [ ]:
is_nan_check(grid1)

shop_id  is nan count  0  ; mean  0.0
item_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
target  is nan count  0  ; mean  0.0
shop_type_lag_1  is nan count  2484728  ; mean  0.2239056080031633
class1_lag_1  is nan count  2484728  ; mean  0.2239056080031633
games_all_lag_1  is nan count  2484728  ; mean  0.2239056080031633
shop_rev_lag_1  is nan count  0  ; mean  0.0
std_price_month_lag_1  is nan count  10573700  ; mean  0.9528249077335821
city_code_lag_1  is nan count  2484728  ; mean  0.2239056080031633
rev_per_item_lag_1  is nan count  0  ; mean  0.0
mean_price_month_lag_1  is nan count  9570563  ; mean  0.8624295003105285
target_lag_1  is nan count  0  ; mean  0.0
max_price_month_lag_1  is nan count  9570563  ; mean  0.8624295003105285
min_price_month_lag_1  is nan count  9570563  ; mean  0.8624295003105285
class4_lag_1  is nan count  2484728  ; mean  0.2239056080031633
class2_lag_1  is nan count  2484728  ; mean  0.2239056080031633
shop_type_lag_2  i

In [ ]:
print(history_shop)
grid1=pd.merge(grid1,history_shop, how='left', on=['item_id','shop_id'])


                 item_shop_start  item_shop_end
item_id shop_id                                
0       54                    20             20
1       55                    15             21
2       54                    19             22
3       54                    18             19
4       54                    20             20
...                          ...            ...
22168   12                     0              8
        16                     1              1
        42                     1              1
        43                     2              2
22169   25                    14             14

[521611 rows x 2 columns]


In [ ]:
#we don't know in which shops the item will be sold, so value 0 means 'no history' 
grid1['shop_history']=(grid1[ 'date_block_num']-grid1['item_shop_start']).clip(0,35)

grid1['shop_history']=grid1['shop_history'].fillna(0)
grid1=grid1.drop(['item_shop_start','item_shop_end'],axis=1)
is_nan_check(grid1,cols=['shop_history'])

shop_history  is nan count  0  ; mean  0.0


In [ ]:
print(grid1)
grid1.loc[ grid1.item_id==15757].groupby('date_block_num').agg(t=('target','sum'))

          shop_id  item_id  date_block_num  target  shop_type_lag_1  \
0              59    22154               0     1.0              NaN   
1              59     2552               0     0.0              NaN   
2              59     2554               0     0.0              NaN   
3              59     2555               0     0.0              NaN   
4              59     2564               0     0.0              NaN   
...           ...      ...             ...     ...              ...   
11097207       45    18454              34     1.0              4.0   
11097208       45    16188              34     1.0              4.0   
11097209       45    15757              34     1.0              4.0   
11097210       45    19648              34     1.0              4.0   
11097211       45      969              34     1.0              4.0   

          class1_lag_1  games_all_lag_1  shop_rev_lag_1  \
0                  NaN              NaN             0.0   
1                  NaN       

,t
date_block_num,
0,14.0
1,10.0
2,9.0
3,7.0
4,9.0
5,11.0
6,14.0
7,19.0
8,15.0


In [ ]:
cols=list(grid1.columns.values)

for col in cols:
     
    grid1[col].replace(np.inf, 0, inplace=True) 
print(grid1.shop_rev_lag_1)

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
           ... 
11097207    0.0
11097208    0.0
11097209    0.0
11097210    0.0
11097211    0.0
Name: shop_rev_lag_1, Length: 11097212, dtype: float16


In [ ]:
#duct tape
#for col in cols:
     
#    print(grid1.loc[grid1[col]==np.inf] )

In [ ]:
grid1.name="grid1"
#save_csv(grid1)

#del grid1,gb
gc.collect() 

3

In [ ]:
#grid1 = pd.read_csv(path+'grid1.csv',usecols=['shop_id','date_block_num'] , encoding='utf8')

In [ ]:
grid1.loc[grid1['date_block_num']==33].shape 

print(grid1.loc[grid1['date_block_num']==33].shop_id.value_counts())
print(grid1.loc[grid1['date_block_num']==33].shop_id.nunique())

59    5413
58    5413
26    5413
25    5413
24    5413
22    5413
21    5413
20    5413
19    5413
18    5413
16    5413
15    5413
14    5413
12    5413
10    5413
9     5413
7     5413
6     5413
5     5413
4     5413
3     5413
28    5413
31    5413
34    5413
47    5413
57    5413
56    5413
55    5413
53    5413
52    5413
50    5413
49    5413
48    5413
46    5413
35    5413
45    5413
44    5413
42    5413
41    5413
39    5413
38    5413
37    5413
36    5413
2     5413
Name: shop_id, dtype: int64
44


In [ ]:
gc.collect()

0

#Encoding text items by common prefix 1:
###EDA to show it makes sense.
###Using  same prefix for similarity; encoding items.csv.
similarity feature for shop month item should not be computed from sales (need to do it from grid). Otherwise it is a data leak!



In [ ]:
#items_custom=items['item_id','item_name']
def custom_string(mystring):
  #prep names
  #prefix: delete everything except listed symbols
  mystring=re.sub( '^[^0-9a-zA-ZА-Яа-я]+','',mystring)
  #suffix: delete everything except listed symbols
  mystring=re.sub( '[^0-9a-zA-ZА-Яа-я\)\(\]\[]+$','',mystring)
  mystring=re.sub( '[\\\.:,/]+',' ',mystring)
  mystring=' '.join(mystring.split())
  
  return mystring

def custom_dist(str1,str2,eps):
  words=0
  n1=len(str1)
  n2=len(str2)
  pref=commonprefix([str1,str2])
  i=len(pref)
  words= pref.count(" ")
  ans=2*(i-1)*(1+np.log(words+1))/(n1+n2-2)+eps
  return ans ,ans   


def sim_tensor(items ):
  # items_custom =items.'item_name_1' , item_id is index
  
  N=len(items) 
  A=np.zeros((N,N),dtype=np.float16)
  for i in range(N):
    #if i%1000==0:
      #print('i= ',i)
    M=min(N,i+100)
    for j in range(i+1,M):
      str1=items[i]
      str2=items[j]
      
      k=abs(i-j)
      eps=1/(1+30*k)
      
      A[i,j],A[j,i]=custom_dist(str1,str2,eps)
       
  A=A.astype(np.float16)
  return A    


In [ ]:
items_custom=items[['item_id','item_name']]
items_custom ['item_name_1']=items_custom.item_name.apply( custom_string)

In [ ]:
#sanity check
#s=items_custom.iloc[400:410]
#s

In [ ]:
#takes a bit of time
A=sim_tensor(items_custom.item_name_1) 


In [ ]:
#most_similar_5=np.argsort(A[:,:] ,axis=1)[-5:,]   #.argmax(A[:,:] ,axis=1) 

maximums=np. max(A[:,:] ,axis=1) 


In [ ]:
most_similar_1=np.argmax(A[:,:] ,axis=1) 
most_similar_1

array([    2,     0,     0, ..., 22166, 22169, 22168])

In [ ]:
# this shows that most similar overall item is not always at distance 1:
item_id=np.arange(22170)
res=pd.DataFrame({'item_id':item_id,'most_similar_1':most_similar_1})
r=(res.item_id-res.most_similar_1).abs().mean()
r

3.355345060893099

In [ ]:
# shows that there are 'similar items' on average
print('maximums.mean() =',maximums.mean())

maximums.mean() = 1.264


In [ ]:
A[1000:1010,1000:1010] 
#print(A[1000:1010,1000:1010])
#pd.DataFrame(A).to_csv(path+'sim_tensor.csv', encoding='utf8')

array([[0.     , 2.086  , 1.159  , 1.259  , 1.12   , 1.2    , 0.1223 ,
        0.1257 , 0.1295 , 0.1063 ],
       [2.086  , 0.     , 1.191  , 1.284  , 1.138  , 1.219  , 0.1255 ,
        0.1287 , 0.1326 , 0.10834],
       [1.159  , 1.191  , 0.     , 1.866  , 1.671  , 1.773  , 0.1094 ,
        0.11084, 0.11304, 0.09503],
       [1.259  , 1.284  , 1.866  , 0.     , 1.817  , 1.928  , 0.122  ,
        0.12305, 0.1255 , 0.1037 ],
       [1.12   , 1.138  , 1.671  , 1.817  , 0.     , 1.75   , 0.11456,
        0.11206, 0.1125 , 0.0946 ],
       [1.2    , 1.219  , 1.773  , 1.928  , 1.75   , 0.     , 0.1381 ,
        0.1256 , 0.12384, 0.10236],
       [0.1223 , 0.1255 , 0.1094 , 0.122  , 0.11456, 0.1381 , 0.     ,
        1.593  , 1.639  , 1.319  ],
       [0.1257 , 0.1287 , 0.11084, 0.12305, 0.11206, 0.1256 , 1.593  ,
        0.     , 1.922  , 1.368  ],
       [0.1295 , 0.1326 , 0.11304, 0.1255 , 0.1125 , 0.12384, 1.639  ,
        1.922  , 0.     , 1.431  ],
       [0.1063 , 0.10834, 0.09503, 0.

In [ ]:
most_similar_1.shape
gc.collect()

57

In [ ]:
#item_shop_month_pairs=sales[sales.item_cnt_day>0].groupby(['shop_id','date_block_num'], as_index=False).agg(active_item_id=('item_id','unique'))
#CAN'T USE THIS FOR DATA SET: there is a data leak!!!
#Leak: contains info on which item has non-zero target in the given shop
item_shop_month=sales[['shop_id','date_block_num','item_id']]
item_shop_month.drop_duplicates(inplace=True, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
item_shop_month

,shop_id,date_block_num,item_id
0,59,0,22154
1,25,0,2552
2,25,0,2554
3,25,0,2555
4,25,0,2564
...,...,...,...
1823313,45,34,18454
1823314,45,34,16188
1823315,45,34,15757
1823316,45,34,19648


In [ ]:
v=item_shop_month.loc [(item_shop_month.shop_id==59) &	(item_shop_month.date_block_num==33)].item_id
w=list(v.values .astype('int16'))
w.sort() 
print(len(w), '\n',w) 
print(v.head())

500 
 [31, 62, 491, 792, 968, 978, 982, 1010, 1121, 1249, 1315, 1332, 1341, 1364, 1377, 1384, 1387, 1389, 1391, 1484, 1496, 1523, 1538, 1540, 1568, 1578, 1581, 1582, 1583, 1584, 1586, 1814, 1816, 1847, 1865, 1876, 1905, 1971, 1988, 2039, 2047, 2090, 2131, 2182, 2252, 2269, 2283, 2293, 2308, 2422, 2423, 2431, 2445, 2461, 2635, 2652, 2671, 2688, 2698, 2734, 2744, 2808, 2816, 2862, 2918, 2919, 2964, 3014, 3076, 3077, 3329, 3341, 3350, 3351, 3352, 3353, 3421, 3442, 3444, 3445, 3479, 3552, 3556, 3599, 3622, 3624, 3651, 3652, 3676, 3708, 3731, 3732, 3734, 3756, 3774, 3831, 3837, 3838, 3839, 3868, 3917, 3967, 4102, 4178, 4181, 4249, 4251, 4334, 4348, 4349, 4351, 4357, 4367, 4385, 4387, 4394, 4424, 4466, 4478, 4516, 4607, 4678, 4706, 4719, 4806, 4838, 4872, 4885, 4892, 4896, 4904, 4962, 5002, 5039, 5041, 5042, 5043, 5044, 5051, 5065, 5233, 5319, 5352, 5362, 5383, 5489, 5542, 5551, 5583, 5657, 5658, 5659, 5661, 5662, 5666, 5670, 5672, 5675, 5811, 5821, 5822, 5823, 5900, 5913, 5946, 5962, 6057, 

In [ ]:
#This example shows that most similar items often have similar ids. However it does not mean that
#they are sold at the same time or the same stores (so as is this is not very useful)
example=pd.merge(item_shop_month,res,how='left', on='item_id' )
example.loc[(example.date_block_num==32)&(example.shop_id==6)]  

,shop_id,date_block_num,item_id,most_similar_1
1565940,6,32,6023,6022
1565941,6,32,6108,6109
1565942,6,32,6126,6127
1565943,6,32,6165,6162
1565944,6,32,6376,6377
...,...,...,...,...
1569365,6,32,15806,15805
1569366,6,32,15821,15822
1569367,6,32,16450,16451
1569368,6,32,16474,16476


In [ ]:
del item_shop_month
gc.collect()

#Finding a similar item for each item.
We find the most similar item for every item,shop,month from the same shop, previous month which was actually sold in that shop.  

In [ ]:
A_df=pd.DataFrame(A[:,: ])
shop_month=grid_sim[['shop_id','date_block_num' ]].drop_duplicates( ignore_index=True).values
shop_month=shop_month.tolist()

In [ ]:
print(shop_month[100:150])

[[45, 2], [31, 2], [30, 2], [32, 2], [54, 2], [53, 2], [56, 2], [57, 2], [47, 2], [43, 2], [50, 2], [51, 2], [52, 2], [18, 2], [19, 2], [16, 2], [15, 2], [21, 2], [22, 2], [23, 2], [7, 2], [6, 2], [8, 2], [10, 2], [14, 2], [12, 2], [13, 2], [28, 2], [27, 2], [29, 2], [25, 2], [24, 2], [26, 2], [58, 2], [59, 2], [3, 2], [4, 2], [25, 3], [21, 3], [23, 3], [22, 3], [24, 3], [28, 3], [27, 3], [29, 3], [26, 3], [7, 3], [16, 3], [10, 3], [2, 3]]


In [ ]:
 
def pairs(data,A,shop ,month):
  #data=grid1 [['shop_id','date_block_num','item_id','target']]  and such
  # for current month every  months/shop/id combination such that the item was sold somewhere in this month
  curr=data.loc [(data.shop_id==shop) &	(data.date_block_num==month)].item_id 
  #print(curr)
  curr_list=list(curr.values.astype('int16'))
  #for previous month data we only consider items that were actually sold in this shop! 
  prev=data.loc [(data.shop_id==shop) &	(data.date_block_num==month-1)&(data.target>0)].item_id
  prev_list=list(prev.values.astype('int16'))
  print(len(curr_list),len(prev_list))
  local=A.iloc[curr_list,prev_list]
  sim_values=local.max(axis=1)
  print(sim_values.shape)
  sim_ids=local.idxmax(axis=1) 
  res=pd.concat((sim_ids,sim_values),axis=1)
  res.rename(columns={0: "item_id_ref", 1: "similarity"},inplace=True)
  res['item_id']=res.index
  res['shop_id']=shop
  res['date_block_num']=month
  res=res[[	'shop_id'	,'date_block_num','item_id','item_id_ref',	'similarity']]	

  return res


In [ ]:
df1=pairs(grid_sim,A_df,shop =59,month=33)
df1

5413 491
(5413,)


,shop_id,date_block_num,item_id,item_id_ref,similarity
13315,59,33,13315,13339,0.364258
13880,59,33,13880,13894,0.710938
13881,59,33,13881,13894,0.868652
13923,59,33,13923,13894,0.001148
14227,59,33,14227,14229,0.872559
...,...,...,...,...,...
7635,59,33,7635,7723,0.000379
7638,59,33,7638,7723,0.000392
7640,59,33,7640,7723,0.000401
7632,59,33,7632,7723,0.000366


In [ ]:
grid_sim

,shop_id,item_id,date_block_num,target,mean_price_month
0,59,22154,0,1.0,999.0
1,59,2552,0,0.0,NaN
2,59,2554,0,0.0,NaN
3,59,2555,0,0.0,NaN
4,59,2564,0,0.0,NaN
...,...,...,...,...,...
11097207,45,18454,34,1.0,1.0
11097208,45,16188,34,1.0,1.0
11097209,45,15757,34,1.0,1.0
11097210,45,19648,34,1.0,1.0


In [ ]:
#producing prefix similarity pairs for every shop/item each month with previous month
#works slowly
pref_pairs=pd.DataFrame()
for x in shop_month:
  print(x[1])
  if x[1]>0:
    try:
      df1=pairs( grid_sim,A_df,shop =x[0],month=x[1])
      print(x)
      pref_pairs=pd.concat((pref_pairs,df1),axis=0)
    except:
      ValueError
      print('previous doesn\'t exist for shop/month: ',x) 
del df1
gc.collect()      

Streaming output truncated to the last 5000 lines.
8
7876 2232
(7876,)
[57, 8]
8
7876 850
(7876,)
[22, 8]
8
7876 1303
(7876,)
[19, 8]
8
7876 1236
(7876,)
[21, 8]
8
7876 898
(7876,)
[18, 8]
8
7876 766
(7876,)
[17, 8]
8
7876 3012
(7876,)
[25, 8]
8
7876 1043
(7876,)
[24, 8]
8
7876 1777
(7876,)
[6, 8]
8
7876 1198
(7876,)
[7, 8]
8
7876 582
(7876,)
[3, 8]
8
7876 996
(7876,)
[4, 8]
8
7876 512
(7876,)
[2, 8]
8
7876 900
(7876,)
[5, 8]
8
7876 1174
(7876,)
[15, 8]
8
7876 1044
(7876,)
[14, 8]
8
7876 1381
(7876,)
[16, 8]
8
7876 474
(7876,)
[10, 8]
8
7876 501
(7876,)
[12, 8]
8
7876 623
(7876,)
[13, 8]
8
7876 3395
(7876,)
[31, 8]
8
7876 1232
(7876,)
[35, 8]
8
7876 1027
(7876,)
[44, 8]
8
7876 1049
(7876,)
[41, 8]
8
7876 909
(7876,)
[45, 8]
8
7876 839
(7876,)
[37, 8]
8
7876 950
(7876,)
[38, 8]
8
7876 1894
(7876,)
[27, 8]
8
7876 2578
(7876,)
[28, 8]
8
7876 1137
(7876,)
[26, 8]
8
7876 1038
(7876,)
[29, 8]
8
7876 1522
(7876,)
[30, 8]
8
7876 1034
(7876,)
[59, 8]
8
7876 1431
(7876,)
[58, 8]
8
7876 1825
(787

557

In [ ]:
pref_pairs

,shop_id,date_block_num,item_id,item_id_ref,similarity
3880,50,1,3880,3869,0.242065
4128,50,1,4128,4127,2.759766
4124,50,1,4124,4127,0.010986
3879,50,1,3879,3869,0.257324
3885,50,1,3885,3878,0.004738
...,...,...,...,...,...
18454,45,34,18454,18456,1.881836
16188,45,34,16188,16227,1.400391
15757,45,34,15757,15759,0.070435
19648,45,34,19648,19659,0.105591


In [ ]:
pref_pairs=downcast_dtypes(pref_pairs)
#correction to merge item_id_ref with previous month data:
#pref_pairs.date_block_num =pref_pairs.date_block_num-1

original dataframe size 182.43910217285156  MBs
downcasted  dataframe  size 182.43910217285156  MBs
fraction of original memory size:  1.0
shop_id             int16
date_block_num      int16
item_id             int16
item_id_ref         int16
similarity        float16
dtype: object


In [ ]:

#pref_pairs.to_csv(path+'pref_pairs'+'.csv', encoding='utf8',index=False)

In [ ]:
pref_pairs

,shop_id,date_block_num,item_id,item_id_ref,similarity
3880,50,1,3880,3869,0.242065
4128,50,1,4128,4127,2.759766
4124,50,1,4124,4127,0.010986
3879,50,1,3879,3869,0.257324
3885,50,1,3885,3878,0.004738
...,...,...,...,...,...
18454,45,34,18454,18456,1.881836
16188,45,34,16188,16227,1.400391
15757,45,34,15757,15759,0.070435
19648,45,34,19648,19659,0.105591


In [ ]:

grid_item_month_for_pref.loc[ 	(
grid_item_month_for_pref.date_block_num==33)]

,item_id,date_block_num,target_item,item_active_shop_count,mean_item_per_active_shop
228498,13315,33,6.0,5.0,1.200000
228499,13880,33,79.0,29.0,2.724138
228500,13881,33,49.0,22.0,2.227273
228501,13923,33,20.0,16.0,1.250000
228502,14227,33,91.0,33.0,2.757576
...,...,...,...,...,...
233906,7635,33,1.0,1.0,1.000000
233907,7638,33,1.0,1.0,1.000000
233908,7640,33,1.0,1.0,1.000000
233909,7632,33,1.0,1.0,1.000000


In [ ]:
# grid_item_month_for_pref ,pref_pairs
shifted_month=grid_item_month_for_pref.copy()
#shifted_month.drop(cols_,inplace=True,axis=1)

shifted_month.date_block_num=shifted_month.date_block_num+1
#print('shifted_month')
#print(shifted_month.loc[shifted_month.date_block_num==35])
 
pref_item_month=pd.merge(pref_pairs,shifted_month,left_on=['item_id_ref',  'date_block_num' ],right_on=['item_id',  'date_block_num' ], how='left', suffixes=('','_sim'))
ren_cols=['target_item',	'item_active_shop_count',	'mean_item_per_active_shop']
renames={}
for x in ren_cols:
  renames[x]=x+'_sim'
pref_item_month.rename(columns=renames,inplace=True) 
#pref_item_month=pd.merge(pref_item_month,shifted_month,left_on=['item_id_ref',  'date_block_num' ],right_on=['item_id',  'date_block_num' ], how='left', suffixes=('','_sim'))
 

In [ ]:
pref_item_month.drop('item_id_sim',axis=1,inplace=True)
pref_item_month=downcast_dtypes(pref_item_month)
pref_item_month.to_csv(path+'pref_item_month'+'.csv', encoding='utf8',index=False)
#shifted_month.loc[shifted_month.item_id==2552]

In [ ]:
pref_item_month

In [ ]:
#del pref_item_month
#gc.collect()


In [ ]:
grid_sim

,shop_id,item_id,date_block_num,target,mean_price_month,target_lag_1,target_lag_2
0,59,22154,0,1.0,999.0,NaN,NaN
1,59,2552,0,0.0,NaN,NaN,NaN
2,59,2554,0,0.0,NaN,NaN,NaN
3,59,2555,0,0.0,NaN,NaN,NaN
4,59,2564,0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11097207,45,18454,34,1.0,1.0,1.0,0.0
11097208,45,16188,34,1.0,1.0,0.0,0.0
11097209,45,15757,34,1.0,1.0,0.0,0.0
11097210,45,19648,34,1.0,1.0,0.0,0.0


In [ ]:
index_cols=['shop_id' ,'item_id' ,'date_block_num']
grid_sim=create_lags(grid_sim,index_cols, shift_range=[1,22 ], no_lags=['mean_price_month'], drop=False)
grid_sim.drop('target_lag_3',axis=1,inplace=True) 

index_cols =  ['shop_id', 'item_id', 'date_block_num']
['target']


In [ ]:
shifted_month=grid_sim.copy()
shifted_month.date_block_num=shifted_month.date_block_num+1

In [ ]:
shifted_month.loc[shifted_month.date_block_num>34]

,shop_id,item_id,date_block_num,target,mean_price_month,target_lag_1,target_lag_2
10883012,5,5037,35,1.0,1.0,0.0,1.0
10883013,5,5320,35,1.0,1.0,NaN,NaN
10883014,5,5233,35,1.0,1.0,1.0,3.0
10883015,5,5232,35,1.0,1.0,0.0,0.0
10883016,5,5268,35,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...
11097207,45,18454,35,1.0,1.0,1.0,0.0
11097208,45,16188,35,1.0,1.0,0.0,0.0
11097209,45,15757,35,1.0,1.0,0.0,0.0
11097210,45,19648,35,1.0,1.0,0.0,0.0


In [ ]:
pref_item =pd.merge(pref_pairs,shifted_month,left_on=['shop_id',	'item_id_ref',  'date_block_num' ],right_on=['shop_id','item_id',  'date_block_num' ], how='left', suffixes=('','_sim'))

ren_cols=['target','mean_price_month'	,'target_lag_1'	,'target_lag_2']
renames={}
for x in ren_cols:
  renames[x]=x+'_sim'
pref_item .rename(columns=renames,inplace=True)  

In [ ]:
pref_item.drop('item_id_sim',axis=1,inplace=True)
pref_item =downcast_dtypes(pref_item )
pref_item .to_csv(path+'pref_item'+'.csv', encoding='utf8',index=False)

original dataframe size 263.5231475830078  MBs
downcasted  dataframe  size 263.5231475830078  MBs
fraction of original memory size:  1.0
shop_id                   int16
date_block_num            int16
item_id                   int16
item_id_ref               int16
similarity              float16
target_sim              float16
mean_price_month_sim    float16
target_lag_1_sim        float16
target_lag_2_sim        float16
dtype: object


In [ ]:
pref_item

,shop_id,date_block_num,item_id,item_id_ref,similarity,target_sim,mean_price_month_sim,target_lag_1_sim,target_lag_2_sim
0,50,1,3880,3869,0.242065,7.0,2500.0,NaN,NaN
1,50,1,4128,4127,2.759766,3.0,1190.0,NaN,NaN
2,50,1,4124,4127,0.010986,3.0,1190.0,NaN,NaN
3,50,1,3879,3869,0.257324,7.0,2500.0,NaN,NaN
4,50,1,3885,3878,0.004738,1.0,3000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10627843,45,34,18454,18456,1.881836,1.0,229.0,0.0,1.0
10627844,45,34,16188,16227,1.400391,1.0,499.0,4.0,3.0
10627845,45,34,15757,15759,0.070435,1.0,149.0,0.0,0.0
10627846,45,34,19648,19659,0.105591,1.0,399.0,1.0,2.0
